In [1]:
import wandb
wandb.login()

%env WANDB_PROJECT=evaluate_LM_with_rationalization

wandb: Currently logged in as: dengdenghuang (cuhk_lavilab). Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=evaluate_LM_with_rationalization


In [2]:
# import gpt3
import logging
import math
import os

from typing import List, Dict, Any, NewType

InputDataClass = NewType("InputDataClass", Any)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from transformers import (
    T5Config,
    T5ForConditionalGeneration,
    T5Tokenizer,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
    EarlyStoppingCallback
)
from transformers.trainer_utils import EvaluationStrategy
from transformers.integrations import TensorBoardCallback
import transformers
from transformers import Trainer

#from feature_conversion_methods import format_instance

from custom_args import (
    DataTrainingArguments,
    ModelArguments
)
from metrics import evaluate
import torch
import datasets
import git
import time
from datetime import datetime
import sys
from tqdm import trange
import random 
import pandas as pd 
import jsonlines
from copy import deepcopy 

logger = logging.getLogger(__name__)
transformers.logging.set_verbosity_info()
import re
def set_global_logging_level(level=logging.ERROR, prefices=[""]):
    """
    Override logging levels of different modules based on their name as a prefix.
    It needs to be invoked after the modules have been loaded so that their loggers have been initialized.

    Args:
        - level: desired level. e.g. logging.INFO. Optional. Default is logging.ERROR
        - prefices: list of one or more str prefices to match (e.g. ["transformers", "torch"]). Optional.
          Default is `[""]` to match all active loggers.
          The match is a case-sensitive `module_name.startswith(prefix)`
    """
    prefix_re = re.compile(fr'^(?:{ "|".join(prefices) })')
    for name in logging.root.manager.loggerDict:
        if re.match(prefix_re, name):
            logging.getLogger(name).setLevel(level)
set_global_logging_level(logging.ERROR, ["datasets"])


CONFIG_MAPPING = {"t5": T5Config}
MODEL_MAPPING = {"t5": T5ForConditionalGeneration}
TOKENIZER_MAPPING = {"t5": T5Tokenizer}


def set_other_seeds(seed):
    torch.backends.cudnn.benchmark = False
    #torch.backends.cudnn.deterministic = True
    os.environ['PYTHONHASHSEED'] = str(seed)

# inspired by DefaultDataCollator from:
# https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py
# modified to perform batch-level padding.
class SequenceCollator:
    def __init__(self, model, pad_token):
        self.model = model
        self.pad_token_mapping = {
            "labels": -100,
            "attention_mask": 0,
            "decoder_attention_mask": 0,
            "input_ids": pad_token,
        }

        self.columns = [
            "input_ids",
            "attention_mask",
            "labels",
            "decoder_attention_mask",
        ]

    def __call__(self, examples: List[Dict[str, InputDataClass]]) -> Dict[str, torch.Tensor]:
        # re-format inputs for training
        batch = {}
        for key in examples[0].keys():
            if key in self.columns:
                tmp_list = []
                for item in examples:
                    tmp_list.append(item[key])

                # pad lists to max length
                if isinstance(tmp_list[0], list):
                    max_length = max(map(len, tmp_list))
                    tmp_list = [
                        el + [self.pad_token_mapping[key]] * (max_length - len(el))
                        for el in tmp_list
                    ]

                batch[key] = torch.tensor(tmp_list, dtype=torch.long)
        return batch


In [3]:
from collections import defaultdict
import random
"""
Example-to-Feature conversion methods
Modified from
https://github.com/salesforce/cos-e/blob/master/code/generation/train_commonsenseqa_v1.0.py and ""_v1.11.py (identical)
as well as Tensorflow code for WTF?: 
https://github.com/google-research/google-research/blob/master/wt5/wt5/preprocessors.py
"""
# This code is based on https://github.com/allenai/label_rationale_association/blob/main/feature_conversion_methods.py

unified_qa_esnli_label_mapping = {0: 'yes', 1: 'maybe', 2: 'no'}
unified_qa_esnli_label_mapping_upper = {0: 'Yes', 1: 'Maybe', 2: 'No'} 
wt5_esnli_label_mapping = {0: 'entailment', 1: 'neutral', 2: 'contradiction'} 
unified_qa_sbic_label_mapping = {"offensive": 'Yes', "not offensive": 'No'}

def format_instance(
        example,
        tokenizer,
        explanation_sep,
        max_seq_length=None,
        datasource=None,
        io_format=None, 
):
    assert datasource in {"cos_e", "esnli", "sbic", "sensemaking", "ecqa"}

    if datasource in ["cos_e", "ecqa"]:
        input_string, answer_string = cqa_formatting(example, io_format, explanation_sep, datasource)
    elif datasource == "esnli":
        input_string, answer_string = esnli_formatting(example, io_format, explanation_sep)
    elif datasource == 'sbic':
        input_string, answer_string = sbic_formatting(example, io_format, explanation_sep)
    elif datasource == 'sensemaking':
        input_string, answer_string = sensemaking_formatting(example, io_format, explanation_sep)
    else:
        raise ValueError("Unknown task. Currently supported: esnli, cos_e, sbic, sensemaking, ecqa.")
    
    if 'unified' in io_format and 'unifew' not in io_format:
        input_string += '</s>'

    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())

    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())

    encodings = tokenizer.encode_plus(
        input_string,
        max_length=max_seq_length,
        pad_to_max_length=False,
        return_token_type_ids=False,
        return_attention_mask=True,
    )


    # note even with "lm_labels.shift_right()", the decoder attention mask length is still correct since we remove the last token
    dec = tokenizer.encode_plus(
        answer_string,
        max_length=max_seq_length,
        pad_to_max_length=False,
        return_token_type_ids=False,
        return_attention_mask=True,
    )

    encodings["labels"] = dec["input_ids"]
    encodings["decoder_attention_mask"] = dec["attention_mask"]
    encodings["question_encoding"] = encodings["input_ids"]

    #return encodings
    return {**example, **encodings}

#这里很简单 定义好输入输出string就可以的
def cqa_formatting(item, io_format, explanation_sep, datasource):
    question = item["question"]
    answer = item["answer"]
    abstr_expl = item["abstractive_explanation"].lower() if datasource == 'cos_e' else item["explanation"].lower()


    if io_format == 't5_fewshot_infilling_with_choices':
        input_string = f"explain {datasource} question: {question} choice: " + " choice: ".join(item["choices"]) + f" <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_more_natural':
        input_string = f"explain {datasource} question: {question} choice: " + " choice: ".join(item["choices"]) + f" The answer is <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == "squad": 
        input_string = f"explain {datasource} question: {question} context: " + ', '.join(item['choices']) # explain cos_e question: When getting in shape you need to have this in between workouts? context: give up, period of recovery, jogging
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # period of recovery because without a period of recovery you will not get any gains.
    elif io_format == "record": 
        # might not work because cos_e doesn't have a passage 
        input_string = f"explain {datasource} query: {question} entities: " + ', '.join(item['choices']) # explain cos_e query: When getting in shape you need to have this in between workouts? entities: give up, period of recovery, jogging
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # period of recovery because without a period of recovery you will not get any gains.
    elif io_format == 'unifiedqa_matching':
        choice_ids = ['(A)', '(B)', '(C)', '(D)', '(E)']
        input_string = f'explain {question.lower()} \\n'
        for choice_id, choice in zip(choice_ids, item["choices"]):
            input_string += f' {choice_id} {choice.lower()}'
        answer_string = f"{answer.lower()} {explanation_sep} {abstr_expl.lower()}"
        answer_string = answer_string.lower()
    elif io_format == 't5_fewshot_infilling_without_choices_use_refined_expl':
        input_string = f"explain {datasource} question: {question} choice: " + " choice: ".join(item["choices"]) + f" <extra_id_0> {explanation_sep} <extra_id_1>"
        input_string = f"explain {datasource} question: {question} answer: {answer}" + f" {explanation_sep} <extra_id_0>"
        answer_string = f"<extra_id_0> {item['our_explanation']} <extra_id_1>"

    else:
        raise ValueError("The IO format is not supported. Choose `standard` or `masked_cause_generate`.")
    
    return input_string, answer_string


def esnli_formatting(item, io_format, explanation_sep):

    premise = item["premise"]
    hypothesis = item["hypothesis"]
    answer = unified_qa_esnli_label_mapping[item["label"]] if 'unified' in io_format else wt5_esnli_label_mapping[item["label"]]
    abstr_expl = item["explanation_1"].lower() 
    # Dev/test instances have more than one explanation annotated; merge them into one sequence separated by [SEP] 
    for k in [2,3]:
        if f"explanation_{k}" in item and item[f'explanation_{k}']!='': 
            abstr_expl += f" [SEP] {item[f'explanation_{k}'].lower()}"

    if io_format == 'standard':
        input_string = f"explain nli hypothesis: {hypothesis} premise: {premise}"
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"
    elif io_format == 't5_fewshot_infilling':
        input_string = f"explain nli hypothesis: {hypothesis} premise: {premise} <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_more_natural':
        input_string = f"explain nli hypothesis: {hypothesis} premise: {premise} This is <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == "squad": 
        input_string = f"explain nli question: Is this entailment? context: {hypothesis} {premise}"  
        answer_ynm = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer_ynm} {explanation_sep} {abstr_expl}" 
    elif io_format == "squad_endswith_what":
        input_string = f"explain nli question: What is this? context: {hypothesis} {premise}"  
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == "squad_nli_mix": 
        input_string = f"explain nli question: Is this entailment? context: hypothesis: {hypothesis} premise: {premise}"  
        answer_ynm = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer_ynm} {explanation_sep} {abstr_expl}"  
    elif io_format == "squad_nli_mix_endswith_what":  
        input_string = f"explain nli question: What is this? context: hypothesis: {hypothesis} premise: {premise}"  
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"   
    elif io_format == 'unifiedqa_unifew':
        hypothesis = hypothesis.lower().rstrip('.')
        unified_qa_esnli_label_mapping_upper = {0: 'Yes', 1: 'Maybe', 2: 'No'}
        answer = unified_qa_esnli_label_mapping_upper[item["label"]]
        input_string = f'explain {premise} Is {hypothesis}? \\n (A) Yes (B) Maybe (C) No'
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == 'unifiedqa_unifew_nli_mix':
        premise = premise.lower().rstrip('.')
        unified_qa_esnli_label_mapping_upper = {0: 'Yes', 1: 'Maybe', 2: 'No'}
        input_string = f'explain hypothesis: {hypothesis} Is premise: {premise}? \\n (A) Yes (B) Maybe (C) No'
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == 'unifiedqa_ynm': 
        input_string = f'explain is this entailment? \\n {hypothesis.lower()} {premise.lower()}'  
        answer = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_ynm': 
        input_string = f'explain is this entailment? \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}' 
        answer = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_ynm_with_choices': 
        input_string = f'explain is this entailment? \\n (A) yes (B) maybe (C) no \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}'  
        answer = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_what_v2': 
        input_string = f'explain what is this? \\n {hypothesis.lower()} {premise.lower()}'  
        answer = wt5_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_what_v2': 
        input_string = f'explain what is this? \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}'  
        answer = wt5_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_what_with_choices_v2': 
        input_string = f'explain what is this? \\n (A) entailment (B) neutral (C) contradiction \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}'  
        answer = wt5_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"     
    else:
        raise ValueError("The IO format is not supported.")

    return input_string, answer_string


def sbic_formatting(item, io_format, explanation_sep):
    # We pre-processed the SBIC dataset such that we join multiple implied statements with the [SEP] token for dev/test instances 
    # Each annotation in the training split is a separate instance
    post = item['post']
    answer = unified_qa_sbic_label_mapping[item["offensiveYN"]] if 'unified' in io_format else item["offensiveYN"].replace("not offensive", "not_offensive")
    abstr_expl = item["targetStereotype"]
    

    if io_format == 'cola':
        input_string = f"explain sbic sentence: {post}"
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == 't5_fewshot_infilling':
        input_string = f"explain sbic sentence: {post} <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_bool':
        answer_bool = 'True' if item["offensiveYN"] == 'offensive' else 'False'
        input_string = f"explain sbic sentence: {post} It is <extra_id_0> that this is offensive {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_more_natural':
        input_string = f"explain sbic sentence: {post} This is <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 'squad_yn': 
        input_string = f"explain sbic question: Is this offensive? context: {post}" # explain sbic question: Is this offensive? context: It's only happened to you like two times.
        answer_yn = unified_qa_sbic_label_mapping[item["offensiveYN"]] 
        answer_string = f"{answer_yn} {explanation_sep} {abstr_expl}" #  No because this post does not imply anything offensive
    elif io_format == 'squad_yn_with_tags': 
        input_string = f"explain sbic question: Is this offensive? context: post: {post}" # explain sbic question: Is this offensive? context: It's only happened to you like two times.
        answer_yn = unified_qa_sbic_label_mapping[item["offensiveYN"]] 
        answer_string = f"{answer_yn} {explanation_sep} {abstr_expl}" #  No because this post does not imply anything offensive
    elif io_format == 'squad_what': 
        input_string = f"explain sbic question: What is this? context: {post}" # explain sbic question: This is what? context: It's only happened to you like two times.
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")        
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # not_offensive because this post does not imply anything offensive    
    elif io_format == 'squad_what_with_tags': 
        input_string = f"explain sbic question: What is this? context: post: {post}" # explain sbic question: This is what? context: It's only happened to you like two times.
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")        
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # not_offensive because this post does not imply anything offensive    
    elif io_format == 'unified_qa_yn':
        input_string = f'explain is this offensive? \\n {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}" 
    elif io_format == 'unified_qa_yn_with_tags':
        input_string = f'explain is this offensive? \\n post: {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_yn_with_choices':
        input_string = f'explain is this offensive? \\n (A) yes (B) no \\n {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"
    elif io_format == 'unified_qa_yn_with_choices_and_tags':
        input_string = f'explain is this offensive? \\n (A) yes (B) no \\n post: {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_what':
        input_string = f'explain what is this? \\n {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_what_with_tags':
        input_string = f'explain what is this? \\n post: {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_what_with_choices':
        input_string = f'explain what is this? \\n (A) offensive (B) not_offensive \\n {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"
    elif io_format == 'unified_qa_what_with_choices_and_tags':
        input_string = f'explain what is this? \\n (A) offensive (B) not_offensive \\n post: {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_unifew':
        input_string = f"Topic? \\n (A) offensive (B) not_offensive \\n {post}"
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"
    else:
        raise ValueError("The IO format is not supported. Choose `standard` or `masked_cause_generate`.")

    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())
    return input_string, answer_string

def sensemaking_formatting(item, io_format, explanation_sep):
    # TODO: explore whether removing periods makes difference? 
    sent0 = item['sent0']
    sent1 = item['sent1']
    nonsensical_sentence = str(int(item['label'])+1)
    explanation = item['explanation'].lower()

    if io_format == 'copa_with_question':
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} question: nonsensical"
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}"
    elif io_format == 'copa_bool':  
        answer_bool = str(bool(int(item['label']))) # True if choice2 is more nonsensical    
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} Less common is choice2"
        answer_string = f"{answer_bool} {explanation_sep} {explanation}"
    elif io_format == 't5_fewshot_infilling':  
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> choice{nonsensical_sentence} <extra_id_1> {explanation} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_bool':  
        answer_bool = str(bool(int(item['label']))) # True if choice2 is more nonsensical    
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} It is <extra_id_0> that choice2 is less common {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer_bool} <extra_id_1> {explanation} <extra_id_2>"
    elif io_format == "squad_yn": 
        input_string = f"explain sensemaking question: Is choice2 more nonsensical? context: choice1: {sent0} choice2: {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer = "Yes" if bool(int(item['label'])) else "No"
        answer_string = f"{answer} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "squad_yn_no_tags": 
        input_string = f"explain sensemaking question: Is choice2 more nonsensical? context: {sent0} {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer = "Yes" if bool(int(item['label'])) else "No"
        answer_string = f"{answer} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "squad_what": 
        input_string = f"explain sensemaking question: What is more nonsensical? context: choice1: {sent0} choice2: {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "squad_what_no_tags": 
        input_string = f"explain sensemaking question: What is more nonsensical? context: {sent0} {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "record": 
        input_string = f"explain sensemaking query: What is more nonsensical? entities: choice1, choice2 passage: choice1: {sent0} choice2: {sent1}" # explain sensemaking query: What is nonsensical? entities: choice1, choice2 passage: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation.
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}" # choice1 because state flowers are unique to each state.
    elif io_format == 'unifiedqa_yn_with_choices':
        answer = "yes" if bool(int(item['label'])) else "no"
        input_string = f'explain is choice2 more nonsensical? \\n (A) yes (B) no \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"{answer} {explanation_sep} {explanation.lower()}" 
    elif io_format == 'unifiedqa_yn':
        answer = "yes" if bool(int(item['label'])) else "no"
        input_string = f'explain is choice2 more nonsensical? \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"{answer} {explanation_sep} {explanation.lower()}"  
    elif io_format == 'unifiedqa_yn_no_tags':
        answer = "yes" if bool(int(item['label'])) else "no"
        input_string = f'explain is choice2 more nonsensical? \\n {sent0.lower()} {sent1.lower()}'
        answer_string = f"{answer} {explanation_sep} {explanation.lower()}"  
    elif io_format == 'unifiedqa_what_with_choices':
        nonsensical_sentence = str(int(item['label'])+1)
        input_string = f'explain what is more nonsensical? \\n (A) choice1 (B) choice2 \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation.lower()}"  # use " BECAUSE "
    elif io_format == 'unifiedqa_what':
        nonsensical_sentence = str(int(item['label'])+1)
        input_string = f'explain what is more nonsensical? \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation.lower()}"  # use " BECAUSE "
    elif io_format == 'unifiedqa_what_no_tags':
        nonsensical_sentence = str(int(item['label'])+1)
        input_string = f'explain what is more nonsensical? \\n {sent0.lower()} {sent1.lower()}'
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation.lower()}"  # use " BECAUSE "


    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())
    return input_string, answer_string

In [4]:
og_start_time = time.time()

#parser = HfArgumentParser(
#    (ModelArguments, DataTrainingArguments, TrainingArguments)
#)
parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments, TrainingArguments)
)

model_args, data_args, training_args, unused_args = parser.parse_args_into_dataclasses(
    ["--model_type", "t5-3b",
     "--tokenizer_name", "t5-3b",
     "--task_name", "cos_e", 
     "--output_dir", "./cos_e_output_t5_large", 
     "--n_shots", "10",
     "--do_train", "True"], return_remaining_strings=True)
if unused_args != []:
    raise ValueError(f"Received unused arguments: {unused_args}")
# make sure only one dataset split pick if manually specifying evaluation file

if model_args.use_gpt3:
    assert training_args.do_train
    assert not training_args.do_eval
    assert data_args.generations_filepath is None
    if data_args.gpt3_max_eval_size is not None:
        assert data_args.gpt3_max_eval_size <= data_args.fewshot_eval_size
        assert data_args.gpt3_max_eval_size % 2 == 0
        assert data_args.gpt3_max_eval_size % 3 == 0

if data_args.generations_filepath is not None:
    training_args.do_train = False
    training_args.do_eval = False
    if "train" in data_args.generations_filepath:
        data_args.train_predict = True
        data_args.test_predict = False
        data_args.dev_predict = False
    elif "test" in data_args.generations_filepath:
        data_args.train_predict = False
        data_args.test_predict = True
        data_args.dev_predict = False
    elif "validation" in data_args.generations_filepath:
        data_args.train_predict = False
        data_args.test_predict = False
        data_args.dev_predict = True

if not training_args.do_train and data_args.generations_filepath is None:
    if not model_args.pretrained_model_file:
        raise Exception(
            "if not training a model from scratch, must specify a trained model to load for evaluation"
        )

if training_args.do_train:
    # create a save directory and a logfile
    training_args.output_dir = os.path.join(
        training_args.output_dir, datetime.now().strftime("%m%d%y_%H%M%S")
    )
    training_args.logging_dir = training_args.output_dir
    assert not os.path.exists(training_args.output_dir)
    os.makedirs(training_args.output_dir)

    if (
            os.path.exists(training_args.output_dir)
            and os.listdir(training_args.output_dir)
            and training_args.do_train
            and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )
    handlers = [
        logging.FileHandler(os.path.join(training_args.output_dir, "logger.log")),
        logging.StreamHandler(),
    ]
else:
    # don't overwrite existing logfile or create new directory
    training_args.output_dir = model_args.pretrained_model_file
    handlers = [logging.StreamHandler()]

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    handlers=handlers,
)
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    training_args.local_rank,
    training_args.device,
    training_args.n_gpu,
    bool(training_args.local_rank != -1),
    training_args.fp16,
)
logger.info("Save path: %s" % training_args.output_dir)

# get git hash and branch where deployed
repo = git.Repo(search_parent_directories=True)
git_hash = repo.head.object.hexsha
git_branch = repo.active_branch.name
logger.info("Git branch: %s" % git_branch)
logger.info("Git hash: %s" % git_hash)

model_class = "t5"
assert data_args.task_name in {"cos_e", "esnli", "sbic", "sensemaking", "ecqa"}

if training_args.do_train:
    # write command and args to file
    with open(
            os.path.join(training_args.output_dir, "commandline_args.txt"), "w"
    ) as f:
        f.write("Git branch: " + git_branch + "\n")
        f.write("Git hash: " + git_hash + "\n")
        f.write("Command:\n")
        f.write("\n".join(sys.argv[1:]))

# Set seed
set_seed(training_args.seed)
set_other_seeds(training_args.seed)

# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
01/13/2023 01:41:04 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
01/13/2023 01:41:04 - INFO - __main__ -   Save path: ./cos_e_output_t5_large/011323_014104
01/13/2023 01:41:04 - INFO - __main__ -   Git branch: dev
01/13/2023 01:41:04 - INFO - __main__ -   Git hash: 1cbb5c3b4e53baf31cbafc20d9655c63f091f901


In [5]:
training_args.set_device = "cuda:0"
training_args.device

device(type='cuda', index=0)

In [6]:
import logging
logger = logging.getLogger(__name__)
CONFIG_MAPPING = {"t5": T5Config}
MODEL_MAPPING = {"t5": T5ForConditionalGeneration}
TOKENIZER_MAPPING = {"t5": T5Tokenizer}
model_class = "t5"
tokenizer_name = TOKENIZER_MAPPING[model_class]
logger.info("Loading pretrained tokenizer...")
model_args.tokenizer_name='t5-base'
tokenizer = tokenizer_name.from_pretrained(model_args.tokenizer_name)#, cache_dir=model_args.cache_dir)

model = T5ForConditionalGeneration.from_pretrained("t5-base")

01/13/2023 01:41:04 - INFO - __main__ -   Loading pretrained tokenizer...
loading file https://huggingface.co/t5-base/resolve/main/spiece.model from cache at /home/huangyongfeng/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
loading file https://huggingface.co/t5-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/t5-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/t5-base/resolve/main/tokenizer_config.json from cache at None
loading file https://huggingface.co/t5-base/resolve/main/tokenizer.json from cache at /home/huangyongfeng/.cache/huggingface/transformers/90de37880b5ff5ac7ab70ff0bd369f207e9b74133fa153c163d14c5bb0116207.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /ho

In [7]:
data_splits = {'train': None, 'validation': None, 'test': None}
original_data_splits = {'train': None, 'validation': None, 'test': None}  
data_args.io_format="t5_fewshot_infilling_without_choices_use_refined_expl"
data_args

DataTrainingArguments(task_name='cos_e', early_stopping_patience=10, overwrite_cache=False, train_predict=False, test_predict=False, dev_predict=False, version_name='v1.11', generations_filepath=None, n_shots=10, fewshot_eval_size=350, io_format='t5_fewshot_infilling_without_choices_use_refined_expl', explanation_sep='explanation', data_path=None, gpt3_max_eval_size=None)

In [8]:
dataset = datasets.load_dataset(data_args.task_name, data_args.version_name)
train_ids_list=[x['id'] for x in dataset["train"]]
dataset['train'][0].keys()

  0%|          | 0/2 [00:00<?, ?it/s]

dict_keys(['abstractive_explanation', 'choices', 'answer', 'id', 'question', 'extractive_explanation'])

In [9]:
#rationale generation labeled data construction 115
import pandas as pd
scr_csqa_labeled_path="/cognitive_comp/huangyongfeng/evaluate_LM_with_rationalization/few_shot_explanations/data/handwritten_cose_v1.11_examples.csv"
scr_csqa_label_df=pd.read_csv(scr_csqa_labeled_path)
scr_csqa_label_data=datasets.load_dataset('csv', data_files=scr_csqa_labeled_path)
scr_csqa_label_ids_list=[x['id'] for x in scr_csqa_label_data['train']]
scr_csqa_indexs_list=[train_ids_list.index(id_) for id_ in scr_csqa_label_ids_list]
scr_csqa_label_our_explanations_list=[x['our_explanation'] for x in scr_csqa_label_data['train']]
print(scr_csqa_indexs_list)
data_splits={}
data_splits['train']=dataset['train'].select(scr_csqa_indexs_list)

refine_train_data=[]
for kk, (ex,da) in enumerate(zip(scr_csqa_label_our_explanations_list, data_splits['train'])):
#     print(da)
    data_splits['train'][kk]['our_explanation']=ex
    #print(type(data_splits['train'][kk]),data_splits['train'][kk].keys())
    da['our_explanation']=ex
    refine_train_data.append(da)

len(data_splits['train'])

  0%|          | 0/1 [00:00<?, ?it/s]

[6723, 3269, 12, 6020, 7815, 0, 13, 7, 3556, 4587, 3166, 2607, 1999, 332, 8571, 2, 3520, 6, 1425, 2361, 3764, 7728, 8352, 1273, 5191, 496, 11, 8143, 6929, 4552, 4652, 7764, 6736, 401, 147, 8339, 3128, 5556, 8769, 4417, 8778, 7145, 1279, 6297, 7149, 5884, 10, 5747, 6273, 8, 9504, 8057, 1307, 2183, 1699, 88, 3228, 7178, 2140, 4048, 1041, 1, 3, 5868, 111, 7889, 5370, 8886, 2137, 6980, 9117, 1891, 7827, 1484, 14, 3668, 7090, 5, 7817, 6213, 4656, 5074, 7514, 5896, 4, 9, 7991, 58, 1887, 1598, 3994, 2212, 2274, 6830, 8826, 1604, 9008, 2013, 6521, 4793, 4338, 3923, 8091, 4904, 3938, 3900, 8156, 5806, 8363, 1787, 1496, 5753, 6351, 1104, 1222]


115

In [10]:
#rationale unlabeled data construction 991
import pdb
scr_csqa_unlabeled_test_file="/cognitive_comp/huangyongfeng/evaluate_LM_with_rationalization/few_shot_explanations/data/acceptability_annotations/commonsenseqa_test.csv"
fse_csqa_dev_dataset = datasets.load_dataset('csv', data_files=scr_csqa_unlabeled_test_file)
scr_csqa_unlabeled_test_df=pd.read_csv(scr_csqa_unlabeled_test_file)
fse_csqa_dev_data_dict={}
for kk, da in enumerate(fse_csqa_dev_dataset['train']):
    #pdb.set_trace()
    id_=da['Input.id']
    if da['Answer.acceptable']:
        answer_accept=set(da['Answer.acceptable'].split('|'))
    else:
        answer_accept=set()
    explanation_list=[da['Input.explanation_1'], 
                      da['Input.explanation_2'],
                      da['Input.explanation_3'],
                      da['Input.explanation_4'],
                      da['Input.explanation_5']]
    if id_ not in fse_csqa_dev_data_dict.keys():
        fse_csqa_dev_data_dict[id_]={"index":kk,"id":id_,
                                     "question":da["Input.question"],
                                     'answer':da['Input.gold_label'],
                                     "accept_set_list":[answer_accept],
                                     "explanation_list":explanation_list}
        #[[kk, id_, answer_accept, explanation_list]]
    else:
        fse_csqa_dev_data_dict[id_]["accept_set_list"].append(answer_accept)
    if len(fse_csqa_dev_data_dict[id_]["accept_set_list"])==3:
        fse_csqa_dev_data_dict[id_]["common_expl_list"]=[]
        common_accept_expl_sample=set.intersection(fse_csqa_dev_data_dict[id_]["accept_set_list"][0], 
                                                   fse_csqa_dev_data_dict[id_]["accept_set_list"][1], 
                                                   fse_csqa_dev_data_dict[id_]["accept_set_list"][2])
        for idx in list(common_accept_expl_sample):
            idx=int(idx)-1
            fse_csqa_dev_data_dict[id_]["common_expl_list"].append(fse_csqa_dev_data_dict[id_]["explanation_list"][idx])
        
#discriminate 3/3 id
id_accept_expl_list=[]
our_accept_expl_list=[]
id_unaccept_expl_list=[]
for k,v in fse_csqa_dev_data_dict.items():
    accept_set_list=v['accept_set_list']
    assert len(accept_set_list)==3
    #pdb.set_trace()
    common_accept_expl_sample=set.intersection(accept_set_list[0], accept_set_list[1], accept_set_list[2])
    #print(accept_set_list,common_accept_expl_sample)
    #pdb.set_trace()
    if common_accept_expl_sample:
        id_accept_expl_list.append(k)
        our_expl=""
        for idx in list(common_accept_expl_sample):
            idx=int(idx)-1
            if len(v['explanation_list'][idx]) > len(our_expl):
                our_expl = v['explanation_list'][idx]
                our_accept_expl_list.append(our_expl)
    else:
        id_unaccept_expl_list.append(k)

dev_ids_list=[x['id'] for x in dataset['validation']]
dev_accept_indexs_list=[dev_ids_list.index(id_) for id_ in id_accept_expl_list]
dev_unaccpet_indexs_list=[dev_ids_list.index(id_) for id_ in id_unaccept_expl_list]
dev_accept_data=dataset['validation'].select(dev_accept_indexs_list)
dev_unaccept_data=dataset['validation'].select(dev_unaccpet_indexs_list)



new_dev_accept_data=[]
new_dev_unaccept_data=[]
for oexp, da in zip(our_accept_expl_list,dev_accept_data):
    da["our_explanation"]=oexp
    new_dev_accept_data.append(da)
for da in dev_unaccept_data:
    da["our_explanation"]=oexp
    new_dev_unaccept_data.append(da)
        
new_dev_accept_data[0]
        

  0%|          | 0/1 [00:00<?, ?it/s]

{'id': '5b8a3081c3235d62bc77e2d15f3ad454',
 'question': 'A town between two mountains is located in a what?',
 'choices': ['valley', 'hospital', 'state', 'train station', 'michigan'],
 'answer': 'valley',
 'abstractive_explanation': 'valleys are always between two mountains',
 'extractive_explanation': 'A town between two mountains',
 'our_explanation': 'A valley is usually between two mountains, where it is located in the place where the mountains are closest to each other.'}

In [11]:
fse_csqa_dev_data_dict['5b8a3081c3235d62bc77e2d15f3ad454']


{'index': 0,
 'id': '5b8a3081c3235d62bc77e2d15f3ad454',
 'question': 'A town between two mountains is located in a what?',
 'answer': 'valley',
 'accept_set_list': [{'2', '4', '5'}, {'2', '4', '5'}, {'2', '4', '5'}],
 'explanation_list': ['Because all of the mountains are to the left of this town, if one was looking on a map, one would be able to see the town in the valley.',
  'A valley is usually between two mountains, where it is located in the place where the mountains are closest to each other.',
  'Because rivers end in valleys; assuming the river is the one of interest, this sentence is likely referring to the geographical location of the town.',
  'A town in between mountains presumably would be in a valley, in which case it is plausable that it would be surrounded by heights in every direction.',
  'A valley is a low area between two mountains.'],
 'common_expl_list': ['A valley is usually between two mountains, where it is located in the place where the mountains are closest 

In [12]:
# for k,v in fse_csqa_dev_data_dict.items():
#     #print(v.keys())
#     print("******")
#     print(v['question'])
#     print(v['answer'])
#     print(v['common_expl_list'])
#     print("******")    
# for k,v in fse_csqa_train_data_dict.items():
#     #print(v.keys())
#     print("******")
#     print(v['question'])
#     print(v['answer'])
#     print(v['common_expl_list'])
#     print("******") 


In [13]:
#rationale unlabeled data construction 991
import pdb
scr_csqa_unlabeled_train_file="/cognitive_comp/huangyongfeng/evaluate_LM_with_rationalization/few_shot_explanations/data/acceptability_annotations/commonsenseqa_train.csv"
fse_csqa_train_dataset = datasets.load_dataset('csv', data_files=scr_csqa_unlabeled_train_file)
scr_csqa_unlabeled_train_df=pd.read_csv(scr_csqa_unlabeled_train_file)
fse_csqa_train_data_dict={}
for kk, da in enumerate(fse_csqa_train_dataset['train']):
    #pdb.set_trace()
    id_=da['Input.id']
    if da['Answer.acceptable']:
        answer_accept=set(da['Answer.acceptable'].split('|'))
    else:
        answer_accept=set()
    explanation_list=[da['Input.explanation_1'], 
                      da['Input.explanation_2'],
                      da['Input.explanation_3'],
                      da['Input.explanation_4'],
                      da['Input.explanation_5']]
    if id_ not in fse_csqa_train_data_dict.keys():
        fse_csqa_train_data_dict[id_]={"index":kk,"id":id_,
                                     "question":da["Input.question"],
                                     'answer':da['Input.gold_label'],
                                     "accept_set_list":[answer_accept],
                                     "explanation_list":explanation_list}
        #[[kk, id_, answer_accept, explanation_list]]
    else:
        fse_csqa_train_data_dict[id_]["accept_set_list"].append(answer_accept)
        
    if len(fse_csqa_train_data_dict[id_]["accept_set_list"])==3:
        fse_csqa_train_data_dict[id_]["common_expl_list"]=[]
        common_accept_expl_sample=set.intersection(fse_csqa_train_data_dict[id_]["accept_set_list"][0], 
                                                   fse_csqa_train_data_dict[id_]["accept_set_list"][1], 
                                                   fse_csqa_train_data_dict[id_]["accept_set_list"][2])
        for idx in list(common_accept_expl_sample):
            idx=int(idx)-1
            fse_csqa_train_data_dict[id_]["common_expl_list"].append(fse_csqa_train_data_dict[id_]["explanation_list"][idx])

    
#discriminate 3/3 id
id_accept_expl_list=[]
our_accept_expl_list=[]
id_unaccept_expl_list=[]
for k,v in fse_csqa_train_data_dict.items():
    accept_set_list=v['accept_set_list']
    assert len(accept_set_list)==3
    #pdb.set_trace()
    common_accept_expl_sample=set.intersection(accept_set_list[0], accept_set_list[1], accept_set_list[2])
    #print(accept_set_list,common_accept_expl_sample)
    #pdb.set_trace()
    if common_accept_expl_sample:
        id_accept_expl_list.append(k)
        our_expl=""
        for idx in list(common_accept_expl_sample):
            idx=int(idx)-1
            if len(v['explanation_list'][idx]) > len(our_expl):
                our_expl = v['explanation_list'][idx]
                our_accept_expl_list.append(our_expl)
    else:
        id_unaccept_expl_list.append(k)

train_ids_list=[x['id'] for x in dataset['train']]
train_accept_indexs_list=[train_ids_list.index(id_) for id_ in id_accept_expl_list]
train_unaccpet_indexs_list=[train_ids_list.index(id_) for id_ in id_unaccept_expl_list]
train_accept_data=dataset['train'].select(train_accept_indexs_list)
train_unaccept_data=dataset['train'].select(train_unaccpet_indexs_list)



new_train_accept_data=[]
new_train_unaccept_data=[]
for oexp, da in zip(our_accept_expl_list,train_accept_data):
    da["our_explanation"]=oexp
    new_train_accept_data.append(da)
for da in train_unaccept_data:
    da["our_explanation"]=oexp
    new_train_unaccept_data.append(da)
        
new_train_accept_data[0]

  0%|          | 0/1 [00:00<?, ?it/s]

{'id': 'ed53cbea1f21072fab892031b31192d1',
 'question': 'Where can you likely buy many poems?',
 'choices': ['book of poetry',
  'literature book',
  'book store',
  'poetry book',
  'bookshelf'],
 'answer': 'book store',
 'abstractive_explanation': 'book store book',
 'extractive_explanation': 'buy many poems',
 'our_explanation': 'A book store is a place where you can buy books, including poetry books.'}

In [14]:
refine_train_data=refine_train_data
refine_dev_data=new_train_accept_data + new_dev_accept_data
refine_test_data=new_train_unaccept_data + new_dev_unaccept_data

refine_train_ids_list=[x['id'] for x in refine_train_data]
refine_dev_ids_list=[x['id'] for x in refine_dev_data]
refine_test_ids_list=[x['id'] for x in refine_test_data]

set(refine_train_ids_list).intersection(set(refine_dev_ids_list)),set(refine_train_ids_list).intersection(set(refine_test_ids_list))



(set(), set())

In [15]:
our_data_splits={}
our_data_splits['train']=refine_train_data
our_data_splits['dev']=refine_dev_data
our_data_splits['test']=refine_test_data
refine_train_data[0].keys()

dict_keys(['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation', 'our_explanation'])

# shots number

In [16]:
id_list = [
 '71519c19654251cced50c5718fd527c1',
 '44195d210c81c8c550174d0edbd3ad5d',
 '5a5da37b685ec9c6f15228128ce30437',
 'cabdfc174953b4bdb8bdcc89d6592c74',
 'eac3a4080127e8924034651b57dfa289',
 'c0013f31657294b9b5ad819123513ca7',
 '2681697be70cd5425899ed6deb84b18a',
 '47f60340b869552e795fa6782224f3d2',
 '7e8a6eac93147eb52d8151bab7b40003',
 'f34267f498c8a78383e35057fbcfa2ce',
 'f024f923d78e5c0a4c0c4baa3ddccf05',
 'e148c710d0c26849cf1f15293481038b',
 'bdf437d7498815f36c255f64a8e6571c',
 '2282e0c80be28aadadb79a822fec9dac',
 'bdd05e96876342b4b5999709439a7f4e',
 '494b8316614fc64c5c27757aeaffce23_1',
 'c4494b402264250dc70931613d482295',
 'fcfbf207c5586294fdce33b645b613f1',
 '9c583bcbe68989a644433302545fb31e',
 '0e684d779d98e6eecda167c10af42bbb',
 'ac0462d09b9fddd40c3c77aedf48a870',
 '1dbfb41e04a10bdbb085bfe667c0df99',
 'cd0c0ef3b8ce0eb09ff26823f6059be6',
 'e6b67184b99375d05d3f2887f684c684',
 'ae9f6f1e17b3fdd3a86b494933809f45',
 'f149604169936a30b7e4c8d599d604b5',
 'd7c8377953e0175ef57d59f3a8e63a8e',
 '1837216ac7745121b82a6491b2318c09',
 'b5fe238d45e55ca5d802b7823ad390ef',
 'b2fbeb0b48bd5e575af0f7dc24cdd85d',
 '9bdfb101c487ac598f6e4686cf774703',
 '206c7116ee2246180fcc0c5a1d0e7205',
 '5cd7f44bbf730dfae5ef2a362d2e5149',
 '45d0ea781ffd49b07d6eae94d47a8945',
 '89afdea1d76272fda8e1538a7d3b7a3f',
 '38a514fe54108527e1d6e89d9812da93',
 'ebf347934a3e90ea46d3d87726fb5d25',
 '90226a6ecd7308ae1e6220a15ae24bc6',
 '5cae8057e75e3adf5c338a43b3ba97fb',
 '388a363c49e1b0aca8eaf18ccdde0caf',
 '9ed3af82152dfcd7000bf4631b095f71',
 '2d5805d894bba224c5fe91e70b8c0f77',
 'd5aa464a16b72c9b9f3091abd461e3ef',
 '381b11c54be4cc432a95053ca34cbf6b',
 '807a3a2dabdd50ed0c76052fd2612632',
 '5172041c750d22e9a5081cc024de0fa6',
 'd32a45ab3b751110d0578a78e54c0e64',
 '28ca5cbe27774746c34f6ebe84fc6d0c']

chatGPT_rationales = [
    "A cup is a common vessel used for holding and drinking liquids, such as water, coffee, or tea. When you want to get yourself a drink, you might typically get a cup from a cabinet or other storage area in order to hold the drink. A cup is a convenient and practical choice for getting a drink because it is easy to hold and use, and it can hold a reasonable amount of liquid. There are many different types of cups available, including ceramic, glass, plastic, and metal cups, each with their own unique features and benefits. Whether you are at home, at work, or out in public, a cup is a handy item to have on hand when you need to quench your thirst.",
    "A front yard is the lawn that is typically located in front of a house, near the street. The front yard is the part of the property that is visible from the street and is often used for decorative purposes, such as landscaping or gardening. Many people take pride in their front yards and may spend time and resources maintaining and improving them. The front yard is often the first thing that visitors see when they arrive at a house, and it can make a strong impression on them. In contrast, the back yard is typically located behind the house and is often used for more practical purposes, such as outdoor recreation or gardening.",
    "A swimming pool is a recreational water feature that is typically used for swimming and other aquatic activities. While shallow water, such as that found in a kiddie pool or a shallow end of a pool, can be enjoyable for some people, it may not be sufficient for many summer activities. A swimming pool that is deep enough for diving and other activities is more likely to provide more opportunities for summer fun. In addition, a swimming pool that is clean and chlorinated is more likely to be safe and enjoyable for swimming, as it will be free of contaminants and bacteria that can cause illness.",
    "In most places, it is illegal for people under the age of 21 to purchase or consume alcohol. This is because alcohol can have negative effects on the developing brain and body of young people. It can also contribute to risky behaviors and accidents. Therefore, many governments have set a minimum age for alcohol consumption and purchase to help protect the health and safety of young people. If someone is under the age of 21 and attempts to purchase alcohol, they may be denied the sale or face legal consequences. It is important for young people to understand and adhere to the laws and regulations regarding the purchase and consumption of alcohol in their area.",
    "When someone shaves, they typically use a razor to remove hair from their face or body. The process of shaving can leave a lot of hair in the sink, as the hair is trimmed off and falls into the sink as it is being shaved. It is common for people to shave in the bathroom, where a sink is typically located. The sink provides a convenient place to rinse the razor and wash off any excess hair or shaving cream. It is important to clean the sink after shaving to remove any hair and prevent clogs in the drain.",
    "A hamburger is a type of sandwich that consists of a cooked patty of ground meat, typically beef, placed between two slices of bread or a bun. Hamburgers are often served with various toppings, such as lettuce, tomato, onion, cheese, and condiments like ketchup and mustard. However, if someone is suffering from great hunger, they may not care about the toppings and just want to eat the hamburger itself. In this case, the person may be more focused on satisfying their hunger and getting nourishment rather than worrying about additional ingredients or flavors.",
    "If a person notices popcorn and cotton candy on the grass, they are likely at fairgrounds. Fairgrounds are outdoor areas that are used for hosting fairs and other events, such as carnivals, festivals, and exhibitions. These events often feature food stalls and vendors that sell a variety of treats, including popcorn and cotton candy. Popcorn and cotton candy are both popular carnival foods that are often sold at fairs and other events. The presence of these foods on the ground at fairgrounds suggests that a fair or similar event is taking place there.",
    "When snow on a mountain becomes heavy, it can trigger an avalanche. An avalanche is a rapid flow of snow down a slope, often triggered by the weight of the snow or external factors such as a change in temperature. Avalanches can be dangerous and destructive, as they can bury and sweep away anything in their path, including people, buildings, and vehicles. Avalanches can occur on any slope with a steep enough incline and a sufficient amount of snow. They are most common on mountains, but they can also occur in other snowy areas, such as forests or valleys. It is important for people to be aware of the risk of avalanches and to take precautions when traveling in areas prone to avalanches.",
    "A police officer typically reports their work through the use of case reports or incident reports. These reports are written documents that detail the events of a specific case or incident that the officer has worked on. Case reports or incident reports typically include information such as the date and time of the incident, the location, the people involved, and the actions taken by the officer. They may also include witness statements, evidence collected, and any other relevant details. Case reports or incident reports are used by police officers to document their work and provide a record of the incidents they have dealt with. They are also used by supervisors and other authorities to review and evaluate the performance of the officer and the effectiveness of their actions.",
    "Teachers play many roles in the lives of their students, including mentor, counselor, and role model. However, they get the most out of their job by simply teaching children. Teaching children is the primary responsibility of a teacher, and it is what they are trained and paid to do. Teaching involves imparting knowledge, skills, and values to students, and helping them to develop the skills and habits they need to succeed in life. Teachers have the opportunity to make a positive impact on the lives of their students by providing them with a strong foundation in academic subjects and helping them to become curious, engaged learners. Teachers often derive a great deal of satisfaction from helping their students to achieve their potential and succeed in their studies.",
    "A person might go to get counseling for a variety of reasons. One reason is to better understand themselves. Counseling can provide a safe and supportive environment for individuals to explore their thoughts, feelings, and behaviors, and to gain insight into why they act and feel the way they do. Counseling can also help a person to identify and work through any underlying issues or challenges that may be causing them distress or hindering their personal growth. Through counseling, a person can learn more about themselves and how to navigate their emotions and relationships more effectively. Counseling can be particularly helpful for individuals who are struggling with mental health issues, relationship problems, or other personal challenges.",
    "If the kids wanted to cool off, they might go swimming as a way to refresh and beat the heat. Swimming is a fun and enjoyable activity that can provide relief from hot weather and help to cool the body. Swimming can also provide a great workout and has numerous health benefits, such as improving cardiovascular fitness and flexibility. There are many different places where kids can go swimming, such as pools, lakes, rivers, or the ocean. No matter where they go, kids can have fun and cool off while enjoying the water.",
    "When remembering a tragedy in the past, many people may feel depressed. Depression is a common emotional reaction to experiencing a traumatic or stressful event, such as a tragedy. Depression is a mental health disorder characterized by persistent feelings of sadness, hopelessness, and low energy. It can also cause physical symptoms such as changes in appetite and sleep patterns, fatigue, and difficulty concentrating. Remembering a tragedy can be emotionally difficult and may trigger feelings of sadness, grief, and guilt. It is important for people who are struggling with depression to seek support and treatment to help manage their symptoms and improve their mental health.",
    "Death Valley is a national park located in the Mojave Desert, in the state of California, United States. It is situated on the North American continent. The North American continent is one of the seven continents of the world and is located in the western hemisphere. It is the third largest continent in terms of land area, after Asia and Africa, and is home to a diverse range of cultures, languages, and environments. Death Valley is known for its extreme desert climate and is the lowest, hottest, and driest place in North America. It is a popular destination for tourists, who come to see its unique geological features, such as sand dunes, salt flats, and canyons.",
    "When in a serious relationship, understanding better requires effort. Relationships require effort and work to maintain and grow. In order to understand your partner better, you need to be willing to put in the time and effort to communicate with them, listen to their perspectives, and try to see things from their point of view. This may involve asking questions, expressing your own feelings and needs, and being open to feedback and compromise. Understanding your partner better can help to strengthen the bond between you and improve the quality of your relationship. It is important to remember that relationships are dynamic and evolving, and that ongoing communication and effort are necessary to maintain a healthy and fulfilling relationship.",
    "Cats are able to make a variety of vocalizations, including meows, purrs, and growls, as a way of communicating with humans and other animals. Cats use these vocalizations to convey a range of emotions and needs, such as hunger, happiness, or discomfort. Meowing is a common vocalization that cats use to communicate with humans. Cats may meow when they want attention, food, or to be let outside. Cats may also meow when they are feeling anxious or stressed. Although cats cannot talk in the same way that humans do, they are skilled at using their voices to communicate with their owners and other animals.",
    "If you need a vessel that is similar to a battleship but faster, you might consider using a corvette. A corvette is a small, fast warship that is typically used for convoy escort, patrol, and other shallow water duties. Corvettes are generally smaller and less heavily armed than battleships, but they are more agile and have a higher speed. They are typically equipped with a variety of weapons, including guns, missiles, and torpedoes, and are used for defending against enemy ships and aircraft. Corvettes are often used for coastal defense, as well as for patrolling and enforcing maritime laws. They are capable of operating in a variety of environments, including open ocean and littoral (nearshore) waters.",
    "Water in the sky comes from rain clouds. Rain clouds are clouds that form when moist air rises and cools, forming tiny water droplets that condense around particles in the air. As the droplets grow, they eventually become heavy enough to fall from the cloud as rain. Rain clouds are typically gray or white in color and are often accompanied by other types of clouds, such as cumulus or stratocumulus clouds. Rain clouds can form in a variety of ways, such as when warm, moist air from the ground rises and cools, or when moist air from the ocean is carried inland by the wind. Rain clouds can be found in many parts of the world and are a common source of precipitation, especially during the warmer months.",
    "If John rented a hotel room while on a business trip and it was a company-sponsored event, it is possible that the hotel was a resort. Resorts are hotels or vacation properties that offer a variety of amenities and activities for guests, such as swimming pools, golf courses, spas, and restaurants. They are often located in scenic or recreational areas and are designed to provide a relaxing and enjoyable experience for guests. Many resorts offer conference and meeting facilities and are popular destinations for business events and retreats. If John's company sponsored the business trip and provided the hotel accommodations, it is likely that they chose a resort with the amenities and facilities needed to support the event.",
    "An expressway might leave the United States and go to Canada. The United States and Canada are neighboring countries that share a border, and there are several expressways that cross the border between the two countries. Expressways are high-speed, limited-access roads that are designed for fast and efficient travel. They typically have multiple lanes and may include features such as grade separations, interchanges, and controlled access points. Expressways are used for long-distance travel and are often the main routes connecting major cities and regions. If an expressway leaves the United States, it is likely that it is heading towards Canada, which is located to the north of the United States.",
    "If you were walking on a sidewalk near a small corner shop run by John, you would look for it on the street corner. A corner shop is a small retail store that is typically located at the intersection of two streets. It is often a convenience store or a neighborhood shop that sells a variety of goods, such as groceries, household items, and other supplies. Corner shops are often located in busy urban areas, where they are easily accessible to pedestrians and motorists. If you were walking on a sidewalk near a corner shop, you would look for it at the intersection of the street where you are walking and another street. The shop is likely to be located in a building on the corner, with windows and doors facing the street.",
    "One way that a company can efficiently sell things to their customers is through mail order. Mail order is a method of selling goods or services by advertising them in a catalog or other publication and allowing customers to place orders by mail, telephone, or online. Mail order is a convenient and efficient way for companies to reach a wide customer base, as it allows them to sell products to customers who may not be located near their physical store or who may not have the time or inclination to shop in person. Mail order companies typically have a centralized warehouse or distribution center where orders are processed and shipped to customers. They may also offer options such as easy returns and exchanges, as well as customer service support to assist with any questions or issues that customers may have.",
    "If he enjoyed the factory work and liked the idea that he did it for sale, it is likely that he enjoyed producing goods. Factory work typically involves the production of goods, such as products or materials, using machines, tools, and other equipment. Factory workers are responsible for operating and maintaining the equipment and ensuring that the goods being produced meet quality standards. The goods produced in a factory may be sold directly to consumers or to other businesses, and may be used for a variety of purposes, such as manufacturing, construction, or distribution. If a person enjoys factory work, they may find satisfaction in the idea that they are contributing to the production of goods that will be sold and used by others.",
    "If returning to work was easy for Dan after his treatment and he was completely painless, it is likely that he had undergone some form of medical treatment or therapy to address a health issue or injury. Painless indicates that Dan was no longer experiencing pain or discomfort, which may have been a result of the treatment or therapy he received. It is not clear from the information provided what specific health issue or injury Dan was dealing with, or what treatment or therapy he received. However, it is possible that Dan was able to return to work more easily after his treatment because he was feeling better and was no longer experiencing any pain or discomfort.",
    "If index cards were on display at the museum, it is likely that the museum was a library. Libraries are institutions that provide access to a wide range of resources, including books, periodicals, and other printed materials, as well as digital resources such as e-books, databases, and online research tools. Libraries often have exhibits and displays of various types of materials, including rare or unique items, and may have special collections or archives that are of particular interest to researchers or historians. Index cards are small, rectangular pieces of paper that are used for organizing information, such as names, dates, or research notes. They may be used in a variety of settings, including libraries, offices, and schools. If index cards were on display at the museum, it is possible that they were part of a historical exhibit or collection related to the history of libraries or research.",
    "If she was a human in America who wanted to live well, it is likely that the given for her situation was the need for money. In general, money is an important factor in enabling people to live well and meet their basic needs and desires. Money allows people to purchase the goods and services that they need to sustain themselves and their families, such as food, clothing, housing, and healthcare. It also enables people to pursue their interests and hobbies, travel, and engage in various forms of leisure and entertainment. In America, and in many other countries around the world, the cost of living can be high, and people often need to earn and save money in order to achieve a desired standard of living. As such, the need for money is a given for many people in order to live well.",
    "A place that could have one back garden is a house. A garden is a piece of land that is used for growing plants, flowers, vegetables, or other types of vegetation. Gardens can be found in a variety of settings, including residential properties, public parks, and commercial properties. A house is a building that is used as a dwelling for one or more people, and it typically includes living spaces, such as bedrooms, bathrooms, and a kitchen, as well as outdoor areas, such as a front yard, side yard, and back yard. A back yard is a private outdoor space that is typically located behind a house and is often used for recreational or leisure activities, such as gardening, grilling, or playing games. If a house has a back garden, it is likely that the garden is located in the back yard.",
    "People gather at a football stadium to see large men in pads play a game where they run and smash into each other. Football is a popular sport that is played on a rectangular field by two teams of eleven players. The game involves running, passing, and tackling, and players wear pads and other protective gear to reduce the risk of injury. Football is a physical and intense sport, and players often collide with each other at high speeds. Football stadiums are large, open-air facilities that are designed to host football games and other sporting events. They typically have a grass or artificial turf field, seating for spectators, and a variety of amenities, such as concessions stands, restrooms, and VIP areas. Football stadiums are popular places for people to gather and watch football games, and they can be found in cities and towns throughout the world.",
    "One place where someone can get a flute is a musical instrument store. A musical instrument store is a retail establishment that specializes in selling musical instruments and related accessories. They may carry a wide range of instruments, including brass instruments, woodwind instruments, string instruments, percussion instruments, and keyboards. Flutes are woodwind instruments that are played by blowing air into a narrow opening and producing sound by vibrating the lips against a metal or wooden reed. They are popular instruments that are used in a variety of musical styles, including classical, jazz, and folk music. If someone is looking to purchase a flute, they may be able to find a selection of flutes at a musical instrument store. In addition to flutes, musical instrument stores may also offer a variety of other instruments, as well as sheet music, instructional materials, and accessories such as cases, stands, and cleaning supplies.",
    "It is not clear what someone who has a greed for energy would do. The word greed typically refers to an excessive or insatiable desire for wealth, power, or material possessions. It is not clear how this concept would relate to energy or how it might influence someone's actions. It is possible that someone who has a lot of energy or who is very active might have a high appetite for food and might seek out sources of fuel or nutrition to sustain their energy levels. In this case, they might buy food as a way to obtain the energy they need. However, it is not accurate to say that someone who has a greed for energy would necessarily do this. It is important to note that the concept of greed is generally considered to be negative, as it suggests an unhealthy or unsustainable desire for something.",
    "A forklift is a type of industrial vehicle that is used for moving and lifting heavy loads, such as pallets, crates, and boxes. Forklifts are equipped with a hydraulic lift mechanism and a set of prongs or forks that can be used to lift and transport loads. They are commonly used in warehouses and other distribution centers to move and stack materials, such as boxes, onto shelves or other storage areas. If you were using a forklift to stack boxes onto shelves, it is likely that you were in a warehouse or a similar facility. Warehouses are large, commercial buildings that are used for storing and distributing goods. They often have high ceilings, wide aisles, and multiple levels, and they may be equipped with a variety of material handling equipment, such as forklifts, conveyor belts, and cranes, to facilitate the movement and storage of goods.",
    "You are likely to receive mouthwash in a little paper cup at a dentist's office. Dentist's offices are healthcare facilities that are staffed by dentists and other dental professionals, and they are used for the diagnosis and treatment of dental problems and the maintenance of oral health. One common practice at a dentist's office is to rinse the mouth with mouthwash, which is a liquid that is used to freshen breath, kill bacteria, and reduce the risk of gum disease and other oral health issues. Mouthwash is typically used after brushing and flossing to help remove any remaining food particles or plaque from the mouth. It may be dispensed in a little paper cup, which can be used to rinse the mouth and then disposed of after use.",
    "A doctor's office can help you with an illness. A doctor's office is a healthcare facility that is staffed by medical doctors and other healthcare professionals, and it is used for the diagnosis and treatment of medical conditions and the maintenance of overall health. If you are experiencing an illness, such as a cold, flu, or more serious health issue, you may visit a doctor's office to seek medical care and treatment. At a doctor's office, you may receive a physical examination, diagnostic tests, and other forms of medical treatment, depending on the nature of your illness and your specific needs. Doctors are trained to diagnose and treat a wide range of medical conditions, and they can provide you with the care and support you need to recover from your illness and get back to good health.",
    "A bug looking for wildflowers and no human interruption might go to a meadow. A meadow is a grassland area that is characterized by its open, grassy landscape and the presence of wildflowers and other types of vegetation. Meadows can be found in a variety of settings, including rural areas, forests, and parkland, and they are often home to a variety of animal and plant species. Because meadows are relatively undisturbed by human activity, they can be a good place for a bug to find wildflowers and other types of plants without interference from humans. Meadows are often considered to be peaceful, serene places, and they may be attractive to a bug that is seeking a quiet, natural environment in which to explore and forage.",
    "If a bride and groom are taking care of proposals, the likely ceremony is a marriage. A marriage is a formal union between two people that is recognized by law and typically involves a public ceremony and the exchange of vows. A marriage proposal is a request for someone to marry another person, and it is typically made by one partner to the other. The process of planning a marriage often involves taking care of a variety of details and arrangements, such as choosing a venue, selecting a date, inviting guests, and organizing the ceremony and reception. If a bride and groom are taking care of proposals, it is likely that they are in the process of planning their marriage and preparing for the wedding ceremony.",
    "You could put some olives in a jar to prevent them from getting nearby food wet. A jar is a container that is typically made of glass or ceramic and has a wide mouth and a secure, tight-fitting lid. Jars are often used for storing food items, such as olives, pickles, jams, and other types of preserved or dried goods. Because they have a sealed lid, jars can help to keep food fresh and prevent it from coming into contact with air or other substances that might spoil or contaminate it. If you want to prevent olives from getting nearby food wet, you could put them in a jar and close the lid to create an airtight seal. This would help to keep the olives separate from other foods and prevent them from coming into contact with moisture or other liquids.",
    "If the graveyard was zoned by the pastor, it is likely that it was located on church property. Church property refers to land or buildings that are owned by a church or religious organization. This may include the main church building, as well as other structures such as a rectory, school, or community center. Church property may also include a graveyard or cemetery, which is a place where people are buried after they die. Pastors, who are ordained leaders in a church, often play a central role in the management and oversight of church property, and they may be responsible for zoning or designating areas within the property for specific purposes, such as a graveyard or a garden. If the pastor has zoned the graveyard, it is likely that it is located on church property and is being used as a place for burial by members of the church or community.",
    "If a family wanted to live with carpeting, they may have ended up putting it in their houses. Houses are structures that are designed and built to be used as homes, and they typically have a number of rooms that are used for different purposes, such as sleeping, cooking, eating, and relaxing. Carpeting is a type of floor covering that is made of soft, flexible materials, such as wool, nylon, or polypropylene, and it is often used in houses to provide warmth, comfort, and insulation. If a family wanted to live with carpeting, they may have chosen to install carpeting in some or all of the rooms in their houses, in order to create a warm and cozy atmosphere and to improve the overall comfort and livability of their homes.",
    "If a boss likes an employee's ambition, the employee may be put in charge of a project. A project is a specific task or set of tasks that is undertaken in order to achieve a specific goal or objective. Projects may be large or small, complex or simple, and they may involve a wide range of activities, such as research, design, development, testing, and implementation. If an employee is ambitious and demonstrates a strong desire to succeed and make a positive contribution to their organization, their boss may choose to put them in charge of a project, in order to give them the opportunity to demonstrate their skills and abilities and to take on more responsibility. Being put in charge of a project can be a challenging and rewarding experience, and it may provide an employee with the chance to learn new things, develop their skills, and make a meaningful contribution to their organization.",
    "A county highway can be viewed as a line on a map. A map is a graphic representation of an area or region that shows the locations of various features, such as roads, rivers, mountains, and cities. Maps are often used to help people navigate and find their way around unfamiliar places, and they can be very useful for understanding the layout and geography of an area. County highways are roads that are maintained and operated by a county or local government, and they are typically used for local travel and transportation within a county or region. On a map, a county highway may be represented as a line, with a specific color or symbol used to indicate its location and route. By viewing a county highway as a line on a map, it is possible to get a sense of the route and layout of the road, and to use the map to plan a journey or navigate to a specific destination.",
    "After going for a run, someone may feel tired. Running is a form of physical activity that involves moving quickly and continuously on foot, typically by running or jogging. It is a popular form of exercise that can help to improve cardiovascular fitness, build endurance, and strengthen the muscles of the legs and lower body. However, running can also be physically demanding, and it requires a certain level of fitness and stamina to be able to sustain the activity for an extended period of time. As a result, it is common for people to feel tired after going for a run, especially if they are not used to the activity or if they have pushed themselves to run for a longer distance or at a faster pace than usual. Tiredness after a run may be accompanied by other physical sensations, such as muscle fatigue, thirst, or a sense of accomplishment.",
    "If a car was going from Florida to Ohio, it was heading north. Florida is a state in the southeastern region of the United States, and Ohio is a state in the midwestern region of the country. When traveling from Florida to Ohio, a person would need to go north in order to reach their destination. North is a direction that is opposite to south on a map or compass, and it is typically associated with higher latitude, colder climates, and longer nights. When traveling north, a person may encounter different landscapes, weather patterns, and cultural experiences than they would when traveling south, depending on the specific route and the distance traveled. By heading north from Florida to Ohio, a person would be leaving behind the warm, tropical climate of Florida and entering into a more temperate, mid-latitude region with cooler temperatures and a more diverse range of landscapes and ecosystems.",
    "During a typical week, a teacher is likely to spend most of their time working in school. A teacher is a person who is trained and qualified to teach students in a school or educational setting, and their primary job is to provide instruction and guidance to help students learn and develop their skills and knowledge. Teachers may work in a variety of settings, including primary schools, secondary schools, and universities, and they may teach a wide range of subjects, including math, science, English, history, and the arts. In order to fulfill their duties as a teacher, a person is likely to spend most of their time working in a school or educational setting, where they will be responsible for preparing and delivering lessons, grading assignments, communicating with students and parents, and performing other tasks related to their job. Teachers typically work on a schedule that follows the school year, and they may work full-time or part-time, depending on their specific circumstances and the needs of their students.",
    "In a predicament, an animal might choose to fight for its life as an alternative to fleeing. Predicament is a term that refers to a difficult or challenging situation, typically one that involves a choice or decision that needs to be made. When an animal finds itself in a predicament, it may have to decide how to respond in order to survive and protect itself. One option that an animal may have is to flee or run away from the situation in an effort to escape danger or avoid conflict. Another option that an animal may have is to fight for its life, using whatever means are available to defend itself and protect its survival. The choice that an animal makes in a predicament will depend on a variety of factors, including the specific nature of the situation, the animal's size, strength, and abilities, and the resources and options that are available to it.",
    "It is not common for people to fight each other when they find a good deal. A good deal is typically defined as an opportunity to purchase something at a price that is lower than what it is usually worth, or to obtain a product or service that provides more value or benefits than what is expected. When people find a good deal, they may feel excited or pleased about the opportunity to save money or get more for their money, and they may take action to take advantage of the deal. However, it is generally not considered acceptable or appropriate for people to fight each other over a good deal or to engage in physical violence or aggression in order to obtain it. There are many other ways that people can express their interest in a good deal, such as by negotiating with the seller, making an offer, or simply making a purchase if the price is acceptable.",
    "An altar is a platform or structure that is used for religious ceremonies, rituals, or worship. Altars are often found in places of worship, such as temples, churches, synagogues, mosques, and other religious buildings. When a person is standing in front of an altar, they are likely to be in a temple or other place of worship. Altars can be used for a variety of purposes in different religions and spiritual traditions, such as offering sacrifices, making offerings, praying, or performing other religious practices. They may be decorated with symbols, images, or other items that are significant to the religion or spiritual tradition that is being practiced. Altars may also be used as a place for people to meditate, reflect, or seek spiritual guidance.",
    "Joe bought a bunch of grapes because he likes to use them to top his desserts. A dessert is a type of food that is typically served at the end of a meal, and it is usually sweet or rich in flavor. There are many different types of desserts that people enjoy, including cakes, cookies, ice cream, pie, and other sweet treats. One way that people may use grapes as a topping for their desserts is by placing them on top of a fruit pie, tart, or crisp. Grapes can add flavor and texture to a dessert, and they may also be used to decorate a dessert or to add visual appeal. Grapes are a versatile fruit that can be used in many different types of desserts, and they can be eaten fresh, frozen, or preserved in various ways.",
    "Well water is water that is obtained from underground sources, typically through the use of a well. A well is a hole or shaft that is dug into the ground in order to access water that is contained in underground aquifers or other water-bearing formations. Well water is typically used for drinking, irrigation, and other purposes, and it is often considered to be of high quality because it is filtered and purified naturally as it flows through the ground. To obtain well water, a person can dig a well or have one dug by a professional well driller. The depth of the well will depend on the location and the geology of the area, and a pump or other mechanical system may be used to bring the water to the surface.",
]

In [17]:
import glob
import json
import pdb
chatgpt_50_100_rationales_dir = "./chatgpt_rationales/200_250_text003"
file_list = glob.glob("{}/*.json".format(chatgpt_50_100_rationales_dir))
chatgpt_rationales_dict = {}
for file in file_list:
    with open(file, 'r') as f:
        da = json.load(f)
        print(da.keys())
        chatgpt_rationales_dict[da['id']] = da['150_to_200_words']
        #pdb.set_trace()

dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(['stem', 'no_more_50_words', 'id', '150_to_200_words'])
dict_keys(

In [18]:
def list2dict(refine_data):
    refine_data_dict={}
    for key in refine_data[0].keys():
        refine_data_dict[key]=[x[key] for x in refine_data]
    return refine_data_dict

select_refine_train_data = []
for da in refine_train_data:
    #pdb.set_trace()
    if da['id'] in chatgpt_rationales_dict.keys():
        da['our_explanation'] = chatgpt_rationales_dict[da['id']]
        select_refine_train_data.append(da) 
        
select_refine_train_data_dict = list2dict(select_refine_train_data)
our_data_splits['train'] = datasets.Dataset.from_dict(select_refine_train_data_dict).shuffle().select(range(72))

refine_dev_data_dict = list2dict(refine_dev_data)
our_data_splits['dev'] = datasets.Dataset.from_dict(refine_dev_data_dict)

refine_test_data_dict = list2dict(refine_test_data)
our_data_splits['test'] = datasets.Dataset.from_dict(refine_test_data_dict)



In [19]:
import datasets
class SequenceCollator:
    def __init__(self, pad_token):
        # self.pad_token_mapping = {
        #     "lm_labels": -100,
        #     "attention_mask": 0,
        #     "decoder_attention_mask": 0,
        #     "input_ids": pad_token,
        # }
        # self.columns = [
        #     "input_ids",
        #     "attention_mask",
        #     "lm_labels",
        #     "decoder_attention_mask",
        # ]
        self.pad_token_mapping = {
            "labels": -100,
            "attention_mask": 0,
            "decoder_attention_mask": 0,
            "input_ids": pad_token,
        }
        self.columns = [
            "input_ids",
            "attention_mask",
            "labels",
            "decoder_attention_mask",
        ]

    def collate_batch(self, examples):

        # batch inputs for training
        batch = {}
        for key in examples[0].keys():
            if key in self.columns:
                tmp_list = []
                for item in examples:
                    tmp_list.append(item[key])

                # pad lists to max length
                if isinstance(tmp_list[0], list):
                    max_length = max(map(len, tmp_list))
                    tmp_list = [
                        el + [self.pad_token_mapping[key]] * (max_length - len(el))
                        for el in tmp_list
                    ]

                batch[key] = torch.tensor(tmp_list, dtype=torch.long)
        return batch
    
    def __call__(self, examples: List[Dict[str, InputDataClass]]) -> Dict[str, torch.Tensor]:
        # re-format inputs for training
        batch = {}
        for key in examples[0].keys():
            if key in self.columns:
                tmp_list = []
                for item in examples:
                    tmp_list.append(item[key])

                # pad lists to max length
                if isinstance(tmp_list[0], list):
                    max_length = max(map(len, tmp_list))
                    tmp_list = [
                        el + [self.pad_token_mapping[key]] * (max_length - len(el))
                        for el in tmp_list
                    ]

                batch[key] = torch.tensor(tmp_list, dtype=torch.long)
        return batch
# dataset = datasets.load_dataset(data_args.task_name, data_args.version_name)

In [20]:
# seq_collector = SequenceCollator(0)
# train_ds = seq_collector.__call__(dataset['train'])
# train_ds
# dataset['train'][0].keys()
for split in ['train','dev','test']:
    our_data_splits[split] = our_data_splits[split].map(
            lambda x: format_instance(
                x,
                tokenizer,
                data_args.explanation_sep,
                datasource=data_args.task_name,
                io_format=data_args.io_format
            ),
            batched=False,
            load_from_cache_file=False,
        )

  0%|          | 0/72 [00:00<?, ?ex/s]

  0%|          | 0/986 [00:00<?, ?ex/s]

  0%|          | 0/255 [00:00<?, ?ex/s]

In [21]:
chatGPT_rationales = [
    "A cup is a common vessel used for holding and drinking liquids, such as water, coffee, or tea. When you want to get yourself a drink, you might typically get a cup from a cabinet or other storage area in order to hold the drink. A cup is a convenient and practical choice for getting a drink because it is easy to hold and use, and it can hold a reasonable amount of liquid. There are many different types of cups available, including ceramic, glass, plastic, and metal cups, each with their own unique features and benefits. Whether you are at home, at work, or out in public, a cup is a handy item to have on hand when you need to quench your thirst.",
    "A front yard is the lawn that is typically located in front of a house, near the street. The front yard is the part of the property that is visible from the street and is often used for decorative purposes, such as landscaping or gardening. Many people take pride in their front yards and may spend time and resources maintaining and improving them. The front yard is often the first thing that visitors see when they arrive at a house, and it can make a strong impression on them. In contrast, the back yard is typically located behind the house and is often used for more practical purposes, such as outdoor recreation or gardening.",
    "A swimming pool is a recreational water feature that is typically used for swimming and other aquatic activities. While shallow water, such as that found in a kiddie pool or a shallow end of a pool, can be enjoyable for some people, it may not be sufficient for many summer activities. A swimming pool that is deep enough for diving and other activities is more likely to provide more opportunities for summer fun. In addition, a swimming pool that is clean and chlorinated is more likely to be safe and enjoyable for swimming, as it will be free of contaminants and bacteria that can cause illness.",
    "In most places, it is illegal for people under the age of 21 to purchase or consume alcohol. This is because alcohol can have negative effects on the developing brain and body of young people. It can also contribute to risky behaviors and accidents. Therefore, many governments have set a minimum age for alcohol consumption and purchase to help protect the health and safety of young people. If someone is under the age of 21 and attempts to purchase alcohol, they may be denied the sale or face legal consequences. It is important for young people to understand and adhere to the laws and regulations regarding the purchase and consumption of alcohol in their area.",
    "When someone shaves, they typically use a razor to remove hair from their face or body. The process of shaving can leave a lot of hair in the sink, as the hair is trimmed off and falls into the sink as it is being shaved. It is common for people to shave in the bathroom, where a sink is typically located. The sink provides a convenient place to rinse the razor and wash off any excess hair or shaving cream. It is important to clean the sink after shaving to remove any hair and prevent clogs in the drain.",
    "A hamburger is a type of sandwich that consists of a cooked patty of ground meat, typically beef, placed between two slices of bread or a bun. Hamburgers are often served with various toppings, such as lettuce, tomato, onion, cheese, and condiments like ketchup and mustard. However, if someone is suffering from great hunger, they may not care about the toppings and just want to eat the hamburger itself. In this case, the person may be more focused on satisfying their hunger and getting nourishment rather than worrying about additional ingredients or flavors.",
    "If a person notices popcorn and cotton candy on the grass, they are likely at fairgrounds. Fairgrounds are outdoor areas that are used for hosting fairs and other events, such as carnivals, festivals, and exhibitions. These events often feature food stalls and vendors that sell a variety of treats, including popcorn and cotton candy. Popcorn and cotton candy are both popular carnival foods that are often sold at fairs and other events. The presence of these foods on the ground at fairgrounds suggests that a fair or similar event is taking place there.",
    "When snow on a mountain becomes heavy, it can trigger an avalanche. An avalanche is a rapid flow of snow down a slope, often triggered by the weight of the snow or external factors such as a change in temperature. Avalanches can be dangerous and destructive, as they can bury and sweep away anything in their path, including people, buildings, and vehicles. Avalanches can occur on any slope with a steep enough incline and a sufficient amount of snow. They are most common on mountains, but they can also occur in other snowy areas, such as forests or valleys. It is important for people to be aware of the risk of avalanches and to take precautions when traveling in areas prone to avalanches.",
    "A police officer typically reports their work through the use of case reports or incident reports. These reports are written documents that detail the events of a specific case or incident that the officer has worked on. Case reports or incident reports typically include information such as the date and time of the incident, the location, the people involved, and the actions taken by the officer. They may also include witness statements, evidence collected, and any other relevant details. Case reports or incident reports are used by police officers to document their work and provide a record of the incidents they have dealt with. They are also used by supervisors and other authorities to review and evaluate the performance of the officer and the effectiveness of their actions.",
    "Teachers play many roles in the lives of their students, including mentor, counselor, and role model. However, they get the most out of their job by simply teaching children. Teaching children is the primary responsibility of a teacher, and it is what they are trained and paid to do. Teaching involves imparting knowledge, skills, and values to students, and helping them to develop the skills and habits they need to succeed in life. Teachers have the opportunity to make a positive impact on the lives of their students by providing them with a strong foundation in academic subjects and helping them to become curious, engaged learners. Teachers often derive a great deal of satisfaction from helping their students to achieve their potential and succeed in their studies.",
    "A person might go to get counseling for a variety of reasons. One reason is to better understand themselves. Counseling can provide a safe and supportive environment for individuals to explore their thoughts, feelings, and behaviors, and to gain insight into why they act and feel the way they do. Counseling can also help a person to identify and work through any underlying issues or challenges that may be causing them distress or hindering their personal growth. Through counseling, a person can learn more about themselves and how to navigate their emotions and relationships more effectively. Counseling can be particularly helpful for individuals who are struggling with mental health issues, relationship problems, or other personal challenges.",
    "If the kids wanted to cool off, they might go swimming as a way to refresh and beat the heat. Swimming is a fun and enjoyable activity that can provide relief from hot weather and help to cool the body. Swimming can also provide a great workout and has numerous health benefits, such as improving cardiovascular fitness and flexibility. There are many different places where kids can go swimming, such as pools, lakes, rivers, or the ocean. No matter where they go, kids can have fun and cool off while enjoying the water.",
    "When remembering a tragedy in the past, many people may feel depressed. Depression is a common emotional reaction to experiencing a traumatic or stressful event, such as a tragedy. Depression is a mental health disorder characterized by persistent feelings of sadness, hopelessness, and low energy. It can also cause physical symptoms such as changes in appetite and sleep patterns, fatigue, and difficulty concentrating. Remembering a tragedy can be emotionally difficult and may trigger feelings of sadness, grief, and guilt. It is important for people who are struggling with depression to seek support and treatment to help manage their symptoms and improve their mental health.",
    "Death Valley is a national park located in the Mojave Desert, in the state of California, United States. It is situated on the North American continent. The North American continent is one of the seven continents of the world and is located in the western hemisphere. It is the third largest continent in terms of land area, after Asia and Africa, and is home to a diverse range of cultures, languages, and environments. Death Valley is known for its extreme desert climate and is the lowest, hottest, and driest place in North America. It is a popular destination for tourists, who come to see its unique geological features, such as sand dunes, salt flats, and canyons.",
    "When in a serious relationship, understanding better requires effort. Relationships require effort and work to maintain and grow. In order to understand your partner better, you need to be willing to put in the time and effort to communicate with them, listen to their perspectives, and try to see things from their point of view. This may involve asking questions, expressing your own feelings and needs, and being open to feedback and compromise. Understanding your partner better can help to strengthen the bond between you and improve the quality of your relationship. It is important to remember that relationships are dynamic and evolving, and that ongoing communication and effort are necessary to maintain a healthy and fulfilling relationship.",
    "Cats are able to make a variety of vocalizations, including meows, purrs, and growls, as a way of communicating with humans and other animals. Cats use these vocalizations to convey a range of emotions and needs, such as hunger, happiness, or discomfort. Meowing is a common vocalization that cats use to communicate with humans. Cats may meow when they want attention, food, or to be let outside. Cats may also meow when they are feeling anxious or stressed. Although cats cannot talk in the same way that humans do, they are skilled at using their voices to communicate with their owners and other animals.",
    "If you need a vessel that is similar to a battleship but faster, you might consider using a corvette. A corvette is a small, fast warship that is typically used for convoy escort, patrol, and other shallow water duties. Corvettes are generally smaller and less heavily armed than battleships, but they are more agile and have a higher speed. They are typically equipped with a variety of weapons, including guns, missiles, and torpedoes, and are used for defending against enemy ships and aircraft. Corvettes are often used for coastal defense, as well as for patrolling and enforcing maritime laws. They are capable of operating in a variety of environments, including open ocean and littoral (nearshore) waters.",
    "Water in the sky comes from rain clouds. Rain clouds are clouds that form when moist air rises and cools, forming tiny water droplets that condense around particles in the air. As the droplets grow, they eventually become heavy enough to fall from the cloud as rain. Rain clouds are typically gray or white in color and are often accompanied by other types of clouds, such as cumulus or stratocumulus clouds. Rain clouds can form in a variety of ways, such as when warm, moist air from the ground rises and cools, or when moist air from the ocean is carried inland by the wind. Rain clouds can be found in many parts of the world and are a common source of precipitation, especially during the warmer months.",
    "If John rented a hotel room while on a business trip and it was a company-sponsored event, it is possible that the hotel was a resort. Resorts are hotels or vacation properties that offer a variety of amenities and activities for guests, such as swimming pools, golf courses, spas, and restaurants. They are often located in scenic or recreational areas and are designed to provide a relaxing and enjoyable experience for guests. Many resorts offer conference and meeting facilities and are popular destinations for business events and retreats. If John's company sponsored the business trip and provided the hotel accommodations, it is likely that they chose a resort with the amenities and facilities needed to support the event.",
    "An expressway might leave the United States and go to Canada. The United States and Canada are neighboring countries that share a border, and there are several expressways that cross the border between the two countries. Expressways are high-speed, limited-access roads that are designed for fast and efficient travel. They typically have multiple lanes and may include features such as grade separations, interchanges, and controlled access points. Expressways are used for long-distance travel and are often the main routes connecting major cities and regions. If an expressway leaves the United States, it is likely that it is heading towards Canada, which is located to the north of the United States.",
    "If you were walking on a sidewalk near a small corner shop run by John, you would look for it on the street corner. A corner shop is a small retail store that is typically located at the intersection of two streets. It is often a convenience store or a neighborhood shop that sells a variety of goods, such as groceries, household items, and other supplies. Corner shops are often located in busy urban areas, where they are easily accessible to pedestrians and motorists. If you were walking on a sidewalk near a corner shop, you would look for it at the intersection of the street where you are walking and another street. The shop is likely to be located in a building on the corner, with windows and doors facing the street.",
    "One way that a company can efficiently sell things to their customers is through mail order. Mail order is a method of selling goods or services by advertising them in a catalog or other publication and allowing customers to place orders by mail, telephone, or online. Mail order is a convenient and efficient way for companies to reach a wide customer base, as it allows them to sell products to customers who may not be located near their physical store or who may not have the time or inclination to shop in person. Mail order companies typically have a centralized warehouse or distribution center where orders are processed and shipped to customers. They may also offer options such as easy returns and exchanges, as well as customer service support to assist with any questions or issues that customers may have.",
    "If he enjoyed the factory work and liked the idea that he did it for sale, it is likely that he enjoyed producing goods. Factory work typically involves the production of goods, such as products or materials, using machines, tools, and other equipment. Factory workers are responsible for operating and maintaining the equipment and ensuring that the goods being produced meet quality standards. The goods produced in a factory may be sold directly to consumers or to other businesses, and may be used for a variety of purposes, such as manufacturing, construction, or distribution. If a person enjoys factory work, they may find satisfaction in the idea that they are contributing to the production of goods that will be sold and used by others.",
    "If returning to work was easy for Dan after his treatment and he was completely painless, it is likely that he had undergone some form of medical treatment or therapy to address a health issue or injury. Painless indicates that Dan was no longer experiencing pain or discomfort, which may have been a result of the treatment or therapy he received. It is not clear from the information provided what specific health issue or injury Dan was dealing with, or what treatment or therapy he received. However, it is possible that Dan was able to return to work more easily after his treatment because he was feeling better and was no longer experiencing any pain or discomfort.",
    "If index cards were on display at the museum, it is likely that the museum was a library. Libraries are institutions that provide access to a wide range of resources, including books, periodicals, and other printed materials, as well as digital resources such as e-books, databases, and online research tools. Libraries often have exhibits and displays of various types of materials, including rare or unique items, and may have special collections or archives that are of particular interest to researchers or historians. Index cards are small, rectangular pieces of paper that are used for organizing information, such as names, dates, or research notes. They may be used in a variety of settings, including libraries, offices, and schools. If index cards were on display at the museum, it is possible that they were part of a historical exhibit or collection related to the history of libraries or research.",
    "If she was a human in America who wanted to live well, it is likely that the given for her situation was the need for money. In general, money is an important factor in enabling people to live well and meet their basic needs and desires. Money allows people to purchase the goods and services that they need to sustain themselves and their families, such as food, clothing, housing, and healthcare. It also enables people to pursue their interests and hobbies, travel, and engage in various forms of leisure and entertainment. In America, and in many other countries around the world, the cost of living can be high, and people often need to earn and save money in order to achieve a desired standard of living. As such, the need for money is a given for many people in order to live well.",
    "A place that could have one back garden is a house. A garden is a piece of land that is used for growing plants, flowers, vegetables, or other types of vegetation. Gardens can be found in a variety of settings, including residential properties, public parks, and commercial properties. A house is a building that is used as a dwelling for one or more people, and it typically includes living spaces, such as bedrooms, bathrooms, and a kitchen, as well as outdoor areas, such as a front yard, side yard, and back yard. A back yard is a private outdoor space that is typically located behind a house and is often used for recreational or leisure activities, such as gardening, grilling, or playing games. If a house has a back garden, it is likely that the garden is located in the back yard.",
    "People gather at a football stadium to see large men in pads play a game where they run and smash into each other. Football is a popular sport that is played on a rectangular field by two teams of eleven players. The game involves running, passing, and tackling, and players wear pads and other protective gear to reduce the risk of injury. Football is a physical and intense sport, and players often collide with each other at high speeds. Football stadiums are large, open-air facilities that are designed to host football games and other sporting events. They typically have a grass or artificial turf field, seating for spectators, and a variety of amenities, such as concessions stands, restrooms, and VIP areas. Football stadiums are popular places for people to gather and watch football games, and they can be found in cities and towns throughout the world.",
    "One place where someone can get a flute is a musical instrument store. A musical instrument store is a retail establishment that specializes in selling musical instruments and related accessories. They may carry a wide range of instruments, including brass instruments, woodwind instruments, string instruments, percussion instruments, and keyboards. Flutes are woodwind instruments that are played by blowing air into a narrow opening and producing sound by vibrating the lips against a metal or wooden reed. They are popular instruments that are used in a variety of musical styles, including classical, jazz, and folk music. If someone is looking to purchase a flute, they may be able to find a selection of flutes at a musical instrument store. In addition to flutes, musical instrument stores may also offer a variety of other instruments, as well as sheet music, instructional materials, and accessories such as cases, stands, and cleaning supplies.",
    "It is not clear what someone who has a greed for energy would do. The word greed typically refers to an excessive or insatiable desire for wealth, power, or material possessions. It is not clear how this concept would relate to energy or how it might influence someone's actions. It is possible that someone who has a lot of energy or who is very active might have a high appetite for food and might seek out sources of fuel or nutrition to sustain their energy levels. In this case, they might buy food as a way to obtain the energy they need. However, it is not accurate to say that someone who has a greed for energy would necessarily do this. It is important to note that the concept of greed is generally considered to be negative, as it suggests an unhealthy or unsustainable desire for something.",
    "A forklift is a type of industrial vehicle that is used for moving and lifting heavy loads, such as pallets, crates, and boxes. Forklifts are equipped with a hydraulic lift mechanism and a set of prongs or forks that can be used to lift and transport loads. They are commonly used in warehouses and other distribution centers to move and stack materials, such as boxes, onto shelves or other storage areas. If you were using a forklift to stack boxes onto shelves, it is likely that you were in a warehouse or a similar facility. Warehouses are large, commercial buildings that are used for storing and distributing goods. They often have high ceilings, wide aisles, and multiple levels, and they may be equipped with a variety of material handling equipment, such as forklifts, conveyor belts, and cranes, to facilitate the movement and storage of goods.",
    "You are likely to receive mouthwash in a little paper cup at a dentist's office. Dentist's offices are healthcare facilities that are staffed by dentists and other dental professionals, and they are used for the diagnosis and treatment of dental problems and the maintenance of oral health. One common practice at a dentist's office is to rinse the mouth with mouthwash, which is a liquid that is used to freshen breath, kill bacteria, and reduce the risk of gum disease and other oral health issues. Mouthwash is typically used after brushing and flossing to help remove any remaining food particles or plaque from the mouth. It may be dispensed in a little paper cup, which can be used to rinse the mouth and then disposed of after use.",
    "A doctor's office can help you with an illness. A doctor's office is a healthcare facility that is staffed by medical doctors and other healthcare professionals, and it is used for the diagnosis and treatment of medical conditions and the maintenance of overall health. If you are experiencing an illness, such as a cold, flu, or more serious health issue, you may visit a doctor's office to seek medical care and treatment. At a doctor's office, you may receive a physical examination, diagnostic tests, and other forms of medical treatment, depending on the nature of your illness and your specific needs. Doctors are trained to diagnose and treat a wide range of medical conditions, and they can provide you with the care and support you need to recover from your illness and get back to good health.",
    "A bug looking for wildflowers and no human interruption might go to a meadow. A meadow is a grassland area that is characterized by its open, grassy landscape and the presence of wildflowers and other types of vegetation. Meadows can be found in a variety of settings, including rural areas, forests, and parkland, and they are often home to a variety of animal and plant species. Because meadows are relatively undisturbed by human activity, they can be a good place for a bug to find wildflowers and other types of plants without interference from humans. Meadows are often considered to be peaceful, serene places, and they may be attractive to a bug that is seeking a quiet, natural environment in which to explore and forage.",
    "If a bride and groom are taking care of proposals, the likely ceremony is a marriage. A marriage is a formal union between two people that is recognized by law and typically involves a public ceremony and the exchange of vows. A marriage proposal is a request for someone to marry another person, and it is typically made by one partner to the other. The process of planning a marriage often involves taking care of a variety of details and arrangements, such as choosing a venue, selecting a date, inviting guests, and organizing the ceremony and reception. If a bride and groom are taking care of proposals, it is likely that they are in the process of planning their marriage and preparing for the wedding ceremony.",
    "You could put some olives in a jar to prevent them from getting nearby food wet. A jar is a container that is typically made of glass or ceramic and has a wide mouth and a secure, tight-fitting lid. Jars are often used for storing food items, such as olives, pickles, jams, and other types of preserved or dried goods. Because they have a sealed lid, jars can help to keep food fresh and prevent it from coming into contact with air or other substances that might spoil or contaminate it. If you want to prevent olives from getting nearby food wet, you could put them in a jar and close the lid to create an airtight seal. This would help to keep the olives separate from other foods and prevent them from coming into contact with moisture or other liquids.",
    "If the graveyard was zoned by the pastor, it is likely that it was located on church property. Church property refers to land or buildings that are owned by a church or religious organization. This may include the main church building, as well as other structures such as a rectory, school, or community center. Church property may also include a graveyard or cemetery, which is a place where people are buried after they die. Pastors, who are ordained leaders in a church, often play a central role in the management and oversight of church property, and they may be responsible for zoning or designating areas within the property for specific purposes, such as a graveyard or a garden. If the pastor has zoned the graveyard, it is likely that it is located on church property and is being used as a place for burial by members of the church or community.",
    "If a family wanted to live with carpeting, they may have ended up putting it in their houses. Houses are structures that are designed and built to be used as homes, and they typically have a number of rooms that are used for different purposes, such as sleeping, cooking, eating, and relaxing. Carpeting is a type of floor covering that is made of soft, flexible materials, such as wool, nylon, or polypropylene, and it is often used in houses to provide warmth, comfort, and insulation. If a family wanted to live with carpeting, they may have chosen to install carpeting in some or all of the rooms in their houses, in order to create a warm and cozy atmosphere and to improve the overall comfort and livability of their homes.",
    "If a boss likes an employee's ambition, the employee may be put in charge of a project. A project is a specific task or set of tasks that is undertaken in order to achieve a specific goal or objective. Projects may be large or small, complex or simple, and they may involve a wide range of activities, such as research, design, development, testing, and implementation. If an employee is ambitious and demonstrates a strong desire to succeed and make a positive contribution to their organization, their boss may choose to put them in charge of a project, in order to give them the opportunity to demonstrate their skills and abilities and to take on more responsibility. Being put in charge of a project can be a challenging and rewarding experience, and it may provide an employee with the chance to learn new things, develop their skills, and make a meaningful contribution to their organization.",
    "A county highway can be viewed as a line on a map. A map is a graphic representation of an area or region that shows the locations of various features, such as roads, rivers, mountains, and cities. Maps are often used to help people navigate and find their way around unfamiliar places, and they can be very useful for understanding the layout and geography of an area. County highways are roads that are maintained and operated by a county or local government, and they are typically used for local travel and transportation within a county or region. On a map, a county highway may be represented as a line, with a specific color or symbol used to indicate its location and route. By viewing a county highway as a line on a map, it is possible to get a sense of the route and layout of the road, and to use the map to plan a journey or navigate to a specific destination.",
    "After going for a run, someone may feel tired. Running is a form of physical activity that involves moving quickly and continuously on foot, typically by running or jogging. It is a popular form of exercise that can help to improve cardiovascular fitness, build endurance, and strengthen the muscles of the legs and lower body. However, running can also be physically demanding, and it requires a certain level of fitness and stamina to be able to sustain the activity for an extended period of time. As a result, it is common for people to feel tired after going for a run, especially if they are not used to the activity or if they have pushed themselves to run for a longer distance or at a faster pace than usual. Tiredness after a run may be accompanied by other physical sensations, such as muscle fatigue, thirst, or a sense of accomplishment.",
    "If a car was going from Florida to Ohio, it was heading north. Florida is a state in the southeastern region of the United States, and Ohio is a state in the midwestern region of the country. When traveling from Florida to Ohio, a person would need to go north in order to reach their destination. North is a direction that is opposite to south on a map or compass, and it is typically associated with higher latitude, colder climates, and longer nights. When traveling north, a person may encounter different landscapes, weather patterns, and cultural experiences than they would when traveling south, depending on the specific route and the distance traveled. By heading north from Florida to Ohio, a person would be leaving behind the warm, tropical climate of Florida and entering into a more temperate, mid-latitude region with cooler temperatures and a more diverse range of landscapes and ecosystems.",
    "During a typical week, a teacher is likely to spend most of their time working in school. A teacher is a person who is trained and qualified to teach students in a school or educational setting, and their primary job is to provide instruction and guidance to help students learn and develop their skills and knowledge. Teachers may work in a variety of settings, including primary schools, secondary schools, and universities, and they may teach a wide range of subjects, including math, science, English, history, and the arts. In order to fulfill their duties as a teacher, a person is likely to spend most of their time working in a school or educational setting, where they will be responsible for preparing and delivering lessons, grading assignments, communicating with students and parents, and performing other tasks related to their job. Teachers typically work on a schedule that follows the school year, and they may work full-time or part-time, depending on their specific circumstances and the needs of their students.",
    "In a predicament, an animal might choose to fight for its life as an alternative to fleeing. Predicament is a term that refers to a difficult or challenging situation, typically one that involves a choice or decision that needs to be made. When an animal finds itself in a predicament, it may have to decide how to respond in order to survive and protect itself. One option that an animal may have is to flee or run away from the situation in an effort to escape danger or avoid conflict. Another option that an animal may have is to fight for its life, using whatever means are available to defend itself and protect its survival. The choice that an animal makes in a predicament will depend on a variety of factors, including the specific nature of the situation, the animal's size, strength, and abilities, and the resources and options that are available to it.",
    "It is not common for people to fight each other when they find a good deal. A good deal is typically defined as an opportunity to purchase something at a price that is lower than what it is usually worth, or to obtain a product or service that provides more value or benefits than what is expected. When people find a good deal, they may feel excited or pleased about the opportunity to save money or get more for their money, and they may take action to take advantage of the deal. However, it is generally not considered acceptable or appropriate for people to fight each other over a good deal or to engage in physical violence or aggression in order to obtain it. There are many other ways that people can express their interest in a good deal, such as by negotiating with the seller, making an offer, or simply making a purchase if the price is acceptable.",
    "An altar is a platform or structure that is used for religious ceremonies, rituals, or worship. Altars are often found in places of worship, such as temples, churches, synagogues, mosques, and other religious buildings. When a person is standing in front of an altar, they are likely to be in a temple or other place of worship. Altars can be used for a variety of purposes in different religions and spiritual traditions, such as offering sacrifices, making offerings, praying, or performing other religious practices. They may be decorated with symbols, images, or other items that are significant to the religion or spiritual tradition that is being practiced. Altars may also be used as a place for people to meditate, reflect, or seek spiritual guidance.",
    "Joe bought a bunch of grapes because he likes to use them to top his desserts. A dessert is a type of food that is typically served at the end of a meal, and it is usually sweet or rich in flavor. There are many different types of desserts that people enjoy, including cakes, cookies, ice cream, pie, and other sweet treats. One way that people may use grapes as a topping for their desserts is by placing them on top of a fruit pie, tart, or crisp. Grapes can add flavor and texture to a dessert, and they may also be used to decorate a dessert or to add visual appeal. Grapes are a versatile fruit that can be used in many different types of desserts, and they can be eaten fresh, frozen, or preserved in various ways.",
    "Well water is water that is obtained from underground sources, typically through the use of a well. A well is a hole or shaft that is dug into the ground in order to access water that is contained in underground aquifers or other water-bearing formations. Well water is typically used for drinking, irrigation, and other purposes, and it is often considered to be of high quality because it is filtered and purified naturally as it flows through the ground. To obtain well water, a person can dig a well or have one dug by a professional well driller. The depth of the well will depend on the location and the geology of the area, and a pump or other mechanical system may be used to bring the water to the surface.",
]

In [22]:
id_list = [
 '71519c19654251cced50c5718fd527c1',
 '44195d210c81c8c550174d0edbd3ad5d',
 '5a5da37b685ec9c6f15228128ce30437',
 'cabdfc174953b4bdb8bdcc89d6592c74',
 'eac3a4080127e8924034651b57dfa289',
 'c0013f31657294b9b5ad819123513ca7',
 '2681697be70cd5425899ed6deb84b18a',
 '47f60340b869552e795fa6782224f3d2',
 '7e8a6eac93147eb52d8151bab7b40003',
 'f34267f498c8a78383e35057fbcfa2ce',
 'f024f923d78e5c0a4c0c4baa3ddccf05',
 'e148c710d0c26849cf1f15293481038b',
 'bdf437d7498815f36c255f64a8e6571c',
 '2282e0c80be28aadadb79a822fec9dac',
 'bdd05e96876342b4b5999709439a7f4e',
 '494b8316614fc64c5c27757aeaffce23_1',
 'c4494b402264250dc70931613d482295',
 'fcfbf207c5586294fdce33b645b613f1',
 '9c583bcbe68989a644433302545fb31e',
 '0e684d779d98e6eecda167c10af42bbb',
 'ac0462d09b9fddd40c3c77aedf48a870',
 '1dbfb41e04a10bdbb085bfe667c0df99',
 'cd0c0ef3b8ce0eb09ff26823f6059be6',
 'e6b67184b99375d05d3f2887f684c684',
 'ae9f6f1e17b3fdd3a86b494933809f45',
 'f149604169936a30b7e4c8d599d604b5',
 'd7c8377953e0175ef57d59f3a8e63a8e',
 '1837216ac7745121b82a6491b2318c09',
 'b5fe238d45e55ca5d802b7823ad390ef',
 'b2fbeb0b48bd5e575af0f7dc24cdd85d',
 '9bdfb101c487ac598f6e4686cf774703',
 '206c7116ee2246180fcc0c5a1d0e7205',
 '5cd7f44bbf730dfae5ef2a362d2e5149',
 '45d0ea781ffd49b07d6eae94d47a8945',
 '89afdea1d76272fda8e1538a7d3b7a3f',
 '38a514fe54108527e1d6e89d9812da93',
 'ebf347934a3e90ea46d3d87726fb5d25',
 '90226a6ecd7308ae1e6220a15ae24bc6',
 '5cae8057e75e3adf5c338a43b3ba97fb',
 '388a363c49e1b0aca8eaf18ccdde0caf',
 '9ed3af82152dfcd7000bf4631b095f71',
 '2d5805d894bba224c5fe91e70b8c0f77',
 'd5aa464a16b72c9b9f3091abd461e3ef',
 '381b11c54be4cc432a95053ca34cbf6b',
 '807a3a2dabdd50ed0c76052fd2612632',
 '5172041c750d22e9a5081cc024de0fa6',
 'd32a45ab3b751110d0578a78e54c0e64',
 '28ca5cbe27774746c34f6ebe84fc6d0c']

In [23]:
# our_wrong_data_splits['dev']=[]
# for x in our_data_splits['dev']:
#     for ii in range(5):
#         new_x={'id':x['id'], 'real_answer':x['answer'], 
#                'answer':x['choices'][ii], 'question':x['question'],
#                'choices':x['choices'],
#                'abstractive_explanation':x['abstractive_explanation'],
#                'extractive_explanation':x['extractive_explanation'],
#                'our_explanation':x['our_explanation']}
#         our_wrong_data_splits['dev'].append(new_x)
#     #print(x)
#     #pdb.set_trace()
# # wandb.init()
# our_wrong_dev_data_dict = list2dict(our_wrong_data_splits['dev'])
# our_wrong_data_splits['dev'] = datasets.Dataset.from_dict(our_wrong_dev_data_dict)

In [24]:
# our_wrong_data_splits['train'][0],our_wrong_data_splits['train'][0].keys()
# our_wrong_data_splits['train'] = our_wrong_data_splits['train'].map(
#             lambda x: format_instance(
#                 x,
#                 tokenizer,
#                 data_args.explanation_sep,
#                 datasource=data_args.task_name,
#                 io_format=data_args.io_format
#             ),
#             batched=False,
#             load_from_cache_file=False,
#         )

In [25]:
# for da in our_wrong_data_splits['dev']:
#     print("*******")
#     print("question: {}".format(da['question']))
#     print("real_answer: {}".format(da['real_answer']))
#     print("answer: {}".format(da['answer']))
#     print("choices: {}".format(da['choices']))
#     print("our_explanation: {}".format(da['our_explanation']))
#     inp_ids = torch.tensor(da["input_ids"], device=model.device).reshape(1, -1)
#     out = model.generate(
#                     inp_ids,
#                     max_length=100,
#                     pad_token_id=tokenizer.pad_token_id,
#                     eos_token_id=tokenizer.eos_token_id,
#                 )
#     skip_special_tokens = False if "infilling" in data_args.io_format else True
#     words = tokenizer.decode(out[0].tolist(), skip_special_tokens=skip_special_tokens)
#     print("generated explanation: {}".format(words))
#     print("#######")

In [26]:
# our_wrong_data_splits['train'][0],our_wrong_data_splits['train'][0].keys()
# our_wrong_data_splits['dev'] = our_wrong_data_splits['dev'].map(
#             lambda x: format_instance(
#                 x,
#                 tokenizer,
#                 data_args.explanation_sep,
#                 datasource=data_args.task_name,
#                 io_format=data_args.io_format
#             ),
#             batched=False,
#             load_from_cache_file=False,
#         )

In [27]:
# for da in our_wrong_data_splits['dev']:
#     print("*******")
#     print("question: {}".format(da['question']))
#     print("real_answer: {}".format(da['real_answer']))
#     print("answer: {}".format(da['answer']))
#     print("choices: {}".format(da['choices']))
#     print("our_explanation: {}".format(da['our_explanation']))
#     inp_ids = torch.tensor(da["input_ids"], device=model.device).reshape(1, -1)
#     out = model.generate(
#                     inp_ids,
#                     max_length=100,
#                     pad_token_id=tokenizer.pad_token_id,
#                     eos_token_id=tokenizer.eos_token_id,
#                 )
#     skip_special_tokens = False if "infilling" in data_args.io_format else True
#     words = tokenizer.decode(out[0].tolist(), skip_special_tokens=skip_special_tokens)
#     print("generated explanation: {}".format(words))
#     print("#######")

In [28]:
# for da in our_wrong_data_splits['train']:
#     print("*******")
#     print("question: {}".format(da['question']))
#     print("real_answer: {}".format(da['real_answer']))
#     print("answer: {}".format(da['answer']))
#     print("choices: {}".format(da['choices']))
#     print("our_explanation: {}".format(da['our_explanation']))
#     inp_ids = torch.tensor(da["input_ids"], device=model.device).reshape(1, -1)
#     out = model.generate(
#                     inp_ids,
#                     max_length=100,
#                     pad_token_id=tokenizer.pad_token_id,
#                     eos_token_id=tokenizer.eos_token_id,
#                 )
#     skip_special_tokens = False if "infilling" in data_args.io_format else True
#     words = tokenizer.decode(out[0].tolist(), skip_special_tokens=skip_special_tokens)
#     print("generated explanation: {}".format(words))
#     print("#######")

In [29]:
# import wandb
# training_args.run_name=""
training_args.logging_steps=3
training_args.save_steps=5
training_args.evaluation_strategy="epoch"
training_args.num_train_epochs=25
training_args.do_eval=True

In [30]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=./cos_e_output_t5_large/011323_014104,
logging_first_step=False,
logging_steps=3,
logging_strategy=IntervalStrategy.STEPS,
lr_scheduler_type=SchedulerType.LINEAR,
max_grad_norm=1.0,
max_steps=-1,
metric_for_best_model=None,
mp_paramet

In [31]:
training_args.per_device_eval_batch_size=8
training_args.per_device_train_batch_size=8
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=./cos_e_output_t5_large/011323_014104,
logging_first_step=False,
logging_steps=3,
logging_strategy=IntervalStrategy.STEPS,
lr_scheduler_type=SchedulerType.LINEAR,
max_grad_norm=1.0,
max_steps=-1,
metric_for_best_model=None,
mp_paramet

In [32]:
# data_splits = {'train': None, 'validation': None, 'test': None}
# original_data_splits = {'train': None, 'validation': None, 'test': None}  
# data_args.io_format="t5_fewshot_infilling_with_choices"
# # Data loading from huggingface's datasets
# if data_args.task_name in {"cos_e", "esnli"}:
#     version_arg = None
#     if data_args.task_name == "cos_e":
#         assert data_args.version_name in {"v1.11", "v1.0"}
#         version_arg = data_args.version_name

#     load_train = True
#     if (not training_args.do_train
#         and not training_args.do_eval
#         and not data_args.train_predict
#     ):
#         # don't load training dataset
#         dataset = {}
#         dataset["train"] = None
#         dataset["validation"] = datasets.load_dataset(
#             data_args.task_name, version_arg, split="validation"
#         )
#         data_splits['validation'] = dataset["validation"]

#         if data_args.task_name == "esnli":
#             dataset["test"] = datasets.load_dataset(data_args.task_name, split="test")
#             data_splits['test'] = dataset["test"]
#         load_train = False
#     else:
#         dataset = datasets.load_dataset(data_args.task_name, version_arg)

#         if data_args.n_shots > 0: # Shots = number of training examples **per label** 
#             if data_args.task_name == 'esnli': # Construct a *balanced* random sample of the size `data_args.n_shots*len(labels)` (for train) or `data_args.fewshot_eval_size` (for eval)
#                 for split in ["train", "validation", "test"]:
#                     split_data = dataset[split]
#                     label_subsets = []
#                     labels = split_data.features['label'].names
#                     sample_size = data_args.n_shots if split == "train" else int(data_args.fewshot_eval_size/len(labels))
#                     if data_args.gpt3_max_eval_size is not None and split != 'train':
#                         assert len(labels) == 3
#                         sample_size = data_args.gpt3_max_eval_size // len(labels)
#                     for label in labels:
#                         # The following is a hack to only run on `neutral` labels of `esnli` to get data for human eval
#                         # if data_args.gpt3_max_eval_size is not None and split != 'train' and label != 'neutral':
#                         #     continue
#                         label_int = split_data.features['label'].str2int(label)
#                         label_set = split_data.filter(lambda example: example['label'] == label_int).shuffle() # all instances of labeled as `label`
#                         label_subset = label_set.select(range(sample_size)) #select `sample_size` random instances labeled as `label`
#                         label_subsets.append(label_subset)
#                     dataset[split] = datasets.concatenate_datasets(label_subsets) #merge all label-specific instances
#             elif data_args.task_name == 'cos_e': 
#                 for split in ["train", "validation"]: 
#                     split_data = dataset[split]
#                     sample_size = data_args.n_shots if split == "train" else int(data_args.fewshot_eval_size) #Shots for QA are not label-specific, i.e., `n_shots` is the training data size
#                     if data_args.gpt3_max_eval_size is not None and split != 'train':
#                         sample_size = data_args.gpt3_max_eval_size
#                     dataset[split] = split_data#.shuffle().select(range(sample_size)) # select `sample_size` random instances
#             else: 
#                 raise ValueError('Only cos_e and esnli are supported by Huggingface datasets.')
#     # Apply method, and format dataset to torch.Tensor outputs
# #     fse_csqa_train_file="/cognitive_comp/huangyongfeng/evaluate_LM_with_rationalization/few_shot_explanations/data/acceptability_annotations/commonsenseqa_train.csv"
# #     fse_csqa_dev_file="/cognitive_comp/huangyongfeng/evaluate_LM_with_rationalization/few_shot_explanations/data/acceptability_annotations/commonsenseqa_test.csv"
# #     fse_csqa_train_dataset = datasets.load_dataset('csv', data_files=fse_csqa_train_file)
# #     fse_csqa_dev_dataset = datasets.load_dataset('csv', data_files=fse_csqa_dev_file)
# #     train_ids_list=[x['id'] for x in data_splits["train"]]
# #     dev_ids_list=[x['id'] for x in data_splits["validation"]]
# #     fse_train_ids_list=[x['Input.id'] for x in fse_csqa_train_dataset['train']]
# #     fse_dev_ids_list=[x['Input.id'] for x in fse_csqa_dev_dataset['train']]
# #     fse_train_indexs_list=[train_ids_list.index(id_) for id_ in fse_train_ids_list]
# #     fse_dev_indexs_list=[dev_ids_list.index(id_) for id_ in fse_dev_ids_list]
# #     print(len(fse_train_indexs_list), len(fse_dev_indexs_list))
# #     # print(fse_train_indexs_list,fse_dev_indexs_list)
# #     fse_data_splits={}
# #     data_splits['train']=data_splits["train"].select(fse_train_indexs_list)
# #     data_splits['validation']=data_splits["validation"].select(fse_train_indexs_list)
#     for split in dataset.keys():
#         if dataset[split] is not None:
#             dataset[split] = dataset[split].map(
#                 lambda x: format_instance(
#                     x,
#                     tokenizer,
#                     data_args.explanation_sep,
#                     datasource=data_args.task_name,
#                     io_format=data_args.io_format
#                 ),
#                 batched=False,
#                 load_from_cache_file=False,
#             )
#     data_splits["train"] = deepcopy(dataset["train"])
#     data_splits["validation"] = deepcopy(dataset["validation"])
#     if data_args.task_name == "esnli":
#         data_splits["test"] = deepcopy(dataset["test"])

#     original_data_splits["train"] = deepcopy(dataset["train"])
#     original_data_splits["validation"] = deepcopy(dataset["validation"])
#     if data_args.task_name == "esnli":
#         original_data_splits["test"] = deepcopy(dataset["test"])

In [33]:
# import pandas as pd
# # new_data_splits={'train': None, 'validation': None}
# # new_data_splits['train']=deepcopy(dataset["train"])
# # new_data_splits['validation']=deepcopy(dataset["validation"])
# fse_csqa_train_file="/cognitive_comp/huangyongfeng/evaluate_LM_with_rationalization/few_shot_explanations/data/acceptability_annotations/commonsenseqa_train.csv"
# fse_csqa_dev_file="/cognitive_comp/huangyongfeng/evaluate_LM_with_rationalization/few_shot_explanations/data/acceptability_annotations/commonsenseqa_test.csv"
# # fse_csqa_train_dataset = datasets.load_dataset('csv', data_files=fse_csqa_train_file)
# # fse_csqa_dev_dataset = datasets.load_dataset('csv', data_files=fse_csqa_dev_file)

# train_df=pd.read_csv(fse_csqa_train_file)

# dev_df=pd.read_csv(fse_csqa_dev_file)

# dev_df


In [34]:
# len(fse_train_ids_list),len(list(set(fse_train_ids_list)))

In [35]:
# train_ids_list=[x['id'] for x in data_splits["train"]]
# dev_ids_list=[x['id'] for x in data_splits["validation"]]
# fse_train_ids_list=[x['Input.id'] for x in fse_csqa_train_dataset['train']]
# fse_dev_ids_list=[x['Input.id'] for x in fse_csqa_dev_dataset['train']]


In [36]:
# fse_train_indexs_list=[train_ids_list.index(id_) for id_ in fse_train_ids_list]
# fse_dev_indexs_list=[dev_ids_list.index(id_) for id_ in fse_dev_ids_list]
# print(len(fse_train_indexs_list), len(fse_dev_indexs_list))
# # print(fse_train_indexs_list,fse_dev_indexs_list)
# fse_data_splits={}
# fse_data_splits['train']=data_splits["train"].select(fse_train_indexs_list)
# fse_data_splits['validation']=data_splits["validation"].select(fse_train_indexs_list)

In [37]:
# fse_csqa_train_dataset['train'][0].keys()

In [38]:
class SequenceCollator:
    def __init__(self, model, pad_token):
        self.model = model
        self.pad_token_mapping = {
            "labels": -100,
            "attention_mask": 0,
            "decoder_attention_mask": 0,
            "input_ids": pad_token,
        }

        self.columns = [
            "input_ids",
            "attention_mask",
            "labels",
            "decoder_attention_mask",
        ]

    def __call__(self, examples: List[Dict[str, InputDataClass]]) -> Dict[str, torch.Tensor]:
        # re-format inputs for training
        batch = {}
        for key in examples[0].keys():
            if key in self.columns:
                tmp_list = []
                for item in examples:
                    tmp_list.append(item[key])

                # pad lists to max length
                if isinstance(tmp_list[0], list):
                    max_length = max(map(len, tmp_list))
                    tmp_list = [
                        el + [self.pad_token_mapping[key]] * (max_length - len(el))
                        for el in tmp_list
                    ]

                batch[key] = torch.tensor(tmp_list, dtype=torch.long)
        return batch

In [39]:
# os.environ["WANDB_DISABLED"] = "True"
if data_args.generations_filepath is None:
    callbacks = [TensorBoardCallback()]
    if data_args.early_stopping_patience > 0:
        callbacks.append(EarlyStoppingCallback(early_stopping_patience=data_args.early_stopping_patience))
        training_args.load_best_model_at_end = True
    else:
        training_args.load_best_model_at_end = False  # use the last model state
    training_args.metric_for_best_model = 'eval_loss'
    training_args.greater_is_better = False
    if training_args.eval_steps is None:
        training_args.evaluation_strategy = EvaluationStrategy.EPOCH
    else:
        training_args.evaluation_strategy = EvaluationStrategy.STEPS

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=our_data_splits['train'],
        eval_dataset=our_data_splits['dev'],
        data_collator=SequenceCollator(
            model=model_class, pad_token=tokenizer.pad_token_id
        ),
        callbacks=callbacks,
    )

# Training. Don't train if it is use_gpt3
if training_args.do_train and not model_args.use_gpt3:
    start_time = time.time()
    trainer.train()
    train_time = time.time() - start_time
    model = trainer.model
    wandb.finish()
else:
    start_time = time.time()
    train_time = time.time() - start_time

You are adding a <class 'transformers.integrations.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
WandbCallback
The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: abstractive_explanation, question_encoding, choices, answer, id, our_explanation, question, extractive_explanation.
***** Running training *****
  Num examples = 72
  Num Epochs = 25
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 225
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss


Saving model checkpoint to ./cos_e_output_t5_large/011323_014104/checkpoint-5
Configuration saved in ./cos_e_output_t5_large/011323_014104/checkpoint-5/config.json
Model weights saved in ./cos_e_output_t5_large/011323_014104/checkpoint-5/pytorch_model.bin
Saving model checkpoint to ./cos_e_output_t5_large/011323_014104/checkpoint-10
Configuration saved in ./cos_e_output_t5_large/011323_014104/checkpoint-10/config.json
Model weights saved in ./cos_e_output_t5_large/011323_014104/checkpoint-10/pytorch_model.bin
Saving model checkpoint to ./cos_e_output_t5_large/011323_014104/checkpoint-15
Configuration saved in ./cos_e_output_t5_large/011323_014104/checkpoint-15/config.json
Model weights saved in ./cos_e_output_t5_large/011323_014104/checkpoint-15/pytorch_model.bin
Saving model checkpoint to ./cos_e_output_t5_large/011323_014104/checkpoint-20
Configuration saved in ./cos_e_output_t5_large/011323_014104/checkpoint-20/config.json
Model weights saved in ./cos_e_output_t5_large/011323_014104

Model weights saved in ./cos_e_output_t5_large/011323_014104/checkpoint-160/pytorch_model.bin
Saving model checkpoint to ./cos_e_output_t5_large/011323_014104/checkpoint-165
Configuration saved in ./cos_e_output_t5_large/011323_014104/checkpoint-165/config.json
Model weights saved in ./cos_e_output_t5_large/011323_014104/checkpoint-165/pytorch_model.bin
Saving model checkpoint to ./cos_e_output_t5_large/011323_014104/checkpoint-170
Configuration saved in ./cos_e_output_t5_large/011323_014104/checkpoint-170/config.json
Model weights saved in ./cos_e_output_t5_large/011323_014104/checkpoint-170/pytorch_model.bin
Saving model checkpoint to ./cos_e_output_t5_large/011323_014104/checkpoint-175
Configuration saved in ./cos_e_output_t5_large/011323_014104/checkpoint-175/config.json
Model weights saved in ./cos_e_output_t5_large/011323_014104/checkpoint-175/pytorch_model.bin
Saving model checkpoint to ./cos_e_output_t5_large/011323_014104/checkpoint-180
Configuration saved in ./cos_e_output_t5

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▆▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,25.0
train/global_step,225


In [40]:
# print(our_data_splits['dev'][0]['our_explanation'])
# inp_ids=torch.tensor(our_data_splits['dev'][0]["input_ids"], device=model.device).reshape(1, -1)
# out = model.generate(
#                     inp_ids,
#                     max_length=100,
#                     pad_token_id=tokenizer.pad_token_id,
#                     eos_token_id=tokenizer.eos_token_id,
#                 )
# skip_special_tokens = False if "infilling" in data_args.io_format else True
# words = tokenizer.decode(out[0].tolist(), skip_special_tokens=skip_special_tokens)
# print(words)

In [41]:
# len(our_data_splits['dev']),len(our_data_splits['train'])

In [42]:
# our_data_splits['dev']

In [43]:
# for da in our_data_splits['train']:
#     print("*******")
#     print("question: {}".format(da['question']))
#     print("answer: {}".format(da['answer']))
#     print("choices: {}".format(da['choices']))
#     print("our_explanation: {}".format(da['our_explanation']))
#     inp_ids = torch.tensor(da["input_ids"], device=model.device).reshape(1, -1)
#     out = model.generate(
#                     inp_ids,
#                     max_length=100,
#                     pad_token_id=tokenizer.pad_token_id,
#                     eos_token_id=tokenizer.eos_token_id,
#                 )
#     skip_special_tokens = False if "infilling" in data_args.io_format else True
#     words = tokenizer.decode(out[0].tolist(), skip_special_tokens=skip_special_tokens)
#     print("generated explanation: {}".format(words))
#     print("#######")

In [44]:
# for da in our_data_splits['dev']:
#     print("*******")
#     print("question: {}".format(da['question']))
#     print("answer: {}".format(da['answer']))
#     print("choices: {}".format(da['choices']))
#     print("our_explanation: {}".format(da['our_explanation']))
#     inp_ids = torch.tensor(da["input_ids"], device=model.device).reshape(1, -1)
#     out = model.generate(
#                     inp_ids,
#                     max_length=100,
#                     pad_token_id=tokenizer.pad_token_id,
#                     eos_token_id=tokenizer.eos_token_id,
#                 )
#     skip_special_tokens = False if "infilling" in data_args.io_format else True
#     words = tokenizer.decode(out[0].tolist(), skip_special_tokens=skip_special_tokens)
#     print("generated explanation: {}".format(words))
#     print("#######")

In [45]:
# good_model = T5ForConditionalGeneration.from_pretrained("./cos_e_output_t5_3b/112022_235539/checkpoint-30")

In [46]:
# for da in our_data_splits['dev']:
#     print("*******")
#     print("question: {}".format(da['question']))
#     print("answer: {}".format(da['answer']))
#     print("choices: {}".format(da['choices']))
#     print("our_explanation: {}".format(da['our_explanation']))
#     inp_ids = torch.tensor(da["input_ids"], device=model.device).reshape(1, -1)
#     out = good_model.generate(
#                     inp_ids,
#                     max_length=100,
#                     pad_token_id=tokenizer.pad_token_id,
#                     eos_token_id=tokenizer.eos_token_id,
#                 )
#     skip_special_tokens = False if "infilling" in data_args.io_format else True
#     words = tokenizer.decode(out[0].tolist(), skip_special_tokens=skip_special_tokens)
#     print("generated explanation: {}".format(words))
#     print("#######")

In [47]:
# train_output = trainer.evaluate(our_data_splits['train'].select([0,1,2,3,4]))
# train_output

In [48]:
# results = {}
# if training_args.do_eval:
#     start_time = time.time()
#     logger.info("*** Evaluate on train set***")
#     logger.info(len(data_splits['train']))
#     train_output = trainer.evaluate(our_data_splits['train'])
#     perplexity = math.exp(train_output["eval_loss"])
#     results["perplexity_train"] = perplexity

#     # repeat
#     logger.info("*** Evaluate on dev set***")
#     logger.info(len(data_splits['validation']))
#     eval_output = trainer.evaluate(data_splits['validation'])
#     perplexity = math.exp(eval_output["eval_loss"])
#     results["perplexity_validation"] = perplexity

# Rationale Discriminator

## Construction of generated rationale and accepted rationale pairs


In [49]:
len(fse_csqa_dev_data_dict)

250

In [50]:
from tqdm import tqdm
fse_csqa_dev_data_dict
fse_csqa_train_data_dict
rationale_pair_dev_data = []
cc=0
good_model = model
for da in tqdm(our_data_splits['dev'], total=len(our_data_splits['dev'])):
    print("*******")
    print("question: {}".format(da['question']))
    print("answer: {}".format(da['answer']))
    print("choices: {}".format(da['choices']))
    #print("our_explanation: {}".format(da['our_explanation']))
    id_ = da['id']
    if id_ in fse_csqa_dev_data_dict.keys():
        common_expl_list = fse_csqa_dev_data_dict[id_]['common_expl_list']
    else:
        common_expl_list = fse_csqa_train_data_dict[id_]['common_expl_list']
    da["common_expl_list"] = common_expl_list
    print("common expl list: {}".format(common_expl_list))
    inp_ids = torch.tensor(da["input_ids"], device=model.device).reshape(1, -1)
    out = good_model.generate(
                    inp_ids,
                    max_length=512,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                )
    skip_special_tokens = False if "infilling" in data_args.io_format else True
    words = tokenizer.decode(out[0].tolist(), skip_special_tokens=skip_special_tokens)
    print("generated explanation: {}".format(words))
    da["generated_explanation"] = words
    print("########")
    rationale_pair_dev_data.append(da)
    cc += 1
    if cc > 200:
        break
    #pdb.set_trace()

  0%|                                                                                                                                                                               | 0/986 [00:00<?, ?it/s]

*******
question: Where can you likely buy many poems?
answer: book store
choices: ['book of poetry', 'literature book', 'book store', 'poetry book', 'bookshelf']
common expl list: ['A book store is a place where you can buy books, including poetry books.', 'A bookstore sells a variety of books, including poetry books; chains of bookstores sometimes specialize in categories such as poetry or literature.']


  0%|▏                                                                                                                                                                  | 1/986 [04:28<73:20:19, 268.04s/it]

generated explanation: <pad> <extra_id_0> Poetry is a form of poetry that is written in a variety of styles and styles. It is a popular form of poetry that is often accompanied by poems, poems, and poems. It is a place to read poetry because it is a place to read them all. It is also a place to read poems in a variety of styles, such as poetry, poetry, and poetry. It is also a place to read and study poetry. It is also a place to buy many poems, such as poetry, poetry magazines, and other materials. It is also a place to buy many poems at a book. It is a great way to read poems in a book. It is a place to read poems in a variety of styles, such as poems that are written in a variety of poems that are written in a variety of styles, such as poetry, poetry, and other things. Poetry is a place to buy poetry in advance. It is a place to buy a book. It is also a place to buy many poems in a variety of poems that are written in a variety of styles, styles, and styles, and that can be written

  0%|▎                                                                                                                                                                  | 2/986 [07:59<64:07:19, 234.59s/it]

...

generated explanation: <pad> <extra_id_0> A massive building is insubstantial if it is not constructed well. It is insubstantial if it is not constructed well. It is a massive building if it is not constructed well. It is a massive building that is not constructed well. It is a massive building that is not constructed well. It is a massive building that is not constructed well. It is a massive building that is not constructed well. It is not constructed well, and it is a huge waste of resources. It is a massive building if it is not constructed well. It is insubstantial if it is not constructed well. It is a building that is not constructed well, and it is not constructed well. It is a huge building that is not constructed well is a massive building that is not constructed well is a huge waste of resources and energy and energy-efficient building. It is not constructed well. It is a waste of energy and energy. It is a huge building that is not constructed well. It is a massive building

 20%|████████████████████████████████▋                                                                                                                                | 200/986 [3:46:06<1:51:04,  8.48s/it]

generated explanation: <pad> <extra_id_0> If you rent a condominium in a glove-shaped state where are you? If you rent a condominium in a glove-shaped state, where are you? If you rent a condominium in a glove-shaped state, where are you located? If you rent a condominium in a glove-shaped state, you are in a state called the glove state. It is located in the state of michigan, it is located in the state of michigan. It is located in the state of Michigan, which is the largest city in the state of Michigan, it is located in the state of Michigan, which is located in the state of Michigan, and it is located in the glove state, it is located in the state of Michigan, it is located in the state of Michigan. The glove-shaped state is located in the state of Michigan, which is located in the state of michigan, it is located in the state of michigan, and it is located in the state of Michigan, where it is located in</s>
########
*******
question: where do children play a game?
answer: family

 20%|████████████████████████████████▍                                                                                                                               | 200/986 [3:46:09<14:48:49, 67.85s/it]

generated explanation: <pad> <extra_id_0> Family room is a great place for children to play a game. Family rooms are a great place for children to have fun and learn new skills. They can also have fun with their parents and friends. They can also play a game in the family room. They can also play a variety of games in the family. They can also play with their friends and family members in a family room. Family room is a great place for children to play a game. Family room is a great place for children to play a game. Family room is a great place to play a game and have fun together. This allows children to play a variety of activities and activities. Family room plays a great way to spend time together as a family room is a great way to keep their memories and memories. Family room is a great way to keep their memories and memories.</s>
########


In [51]:
import json
rationale_pair_save_path = os.path.join("./results", "72shots_cose_t5_base_text003_200_250_rationales_generator_test_rationale_pair.json")
with open(rationale_pair_save_path, 'w') as f:
    json.dump(rationale_pair_dev_data, f)

In [52]:
fse_csqa_dev_data_dict.keys()

dict_keys(['5b8a3081c3235d62bc77e2d15f3ad454', '07f108d5321a66f460685f5c7499ecb2', 'ff1bf2ec835c9df8695ae0cfb5281646', '33ea932a876ac0361c9eefeff1d24e92', '35ad89c198d5d6311a71c993bb7b6cba', 'a2aa95861ef74bf1ecfc55db505e3982', 'af3b9a8b1962cd3bcd19e644d873e7bc', '91e0f4ab62c9d2fd440d73a3f5308d96', 'cfc7fccb8449a2a950c9d2a50991420e', 'edbb57ac2f476679ae547f75ec2bef3e', '8555dd9667d010018961a2f7d1c22704', '8bdbb8caefcc607a9ec7579aa0c87cba', '549cf641318edfc0510fa7c7dbb359e1', 'd6c002d46d9bfa466637cec4a134f332', '3b9ccdcb1c932c46a38e040d3e6c7f5b', '54231f875bb7fe4d3e4afb6eae64387c', '668dc6bce771b10cbf6336f3ec76520a', '900492bd731f8f615ed7c08155737d44', '0b25bbd9e9aa976655e1975e31331709', '1729c737ff92cf558efecde2c6cafc5e', 'cc1a547bdfdcc95e4d632453af14bc96', 'c492b8b9754a181c924c1df19998cbc7', 'd31ee38f67d1173275e120b8ad36039c', '083861fc5ebb9226fff70544f3f83d2b', 'e1744fc698cffb574e5cf4b29a81ce76', '1f492f556fae64f72ce36b6caa242dd0', '27604394ccee83e089f9ffae1883cf07', 'b3dc6d6a5e2f9d7d